In [1]:
#!pip install graphviz --user
#!echo $PYTHONPATH
#!ls -ltr /eos/user/n/nmangane/.local/lib/python2.7/site-packages/
#!export PATH=/eos/user/n/nmangane/.local/lib/python2.7/site-packages/:$PATH
#!ls -ltr | grep .root

In [16]:
from __future__ import print_function
import os, time
import ROOT
import collections
import pprint
import math
#from ruamel.yaml import YAML
from IPython.display import Image, display, SVG
#import graphviz

In [17]:
argsDOTplotJSON = {
    "DefaultPlot":{
        "Type": "DefaultPlot",
        "Name": "DefaultPlotName",
        "Rebin": None,
        "Files": None,
    },
    "DefaultLegend":{
        "Type": "DefaultLegend",
        "Coordinates": [0.75, 0.80, 0.95, 0.90],
        "Categories": {
            "tttt": {"Color": ROOT.kAzure-2,
                     "Names": ["tttt"],
                     "Style": "Fill",
                    },
            "ttbar": {"Color": ROOT.kRed,
                      "Names": ["tt_DL", "tt_SL", "tt_DL-GF", "tt_SL-GF"],
                      "Style": "Fill",
                     },
            "singletop": {"Color": ROOT.kYellow,
                          "Names": ["ST_tW", "ST_tbarW"],
                          "Style": "Fill",
                         },
            "ttH":  {"Color": ROOT.kMagenta,
                     "Names": ["ttH"],
                     "Style": "Fill",
                    },
            "ttVJets": {"Color": ROOT.kViolet,
                        "Names": ["ttWJets", "ttZJets"],
                        "Style": "Fill",
                       },
            "ttultrarare": {"Color": ROOT.kGreen,
                            "Names": ["ttWW", "ttWH", "ttWZ", "ttZZ", "ttZH", "ttHH", "tttJ"],
                            "Style": "Fill",
                           },
            "DY": {"Color": ROOT.kCyan,
                   "Names": ["DYJets_DL"],
                   "Style": "Fill",
                  },
            "Data": {"Color": ROOT.kBlack,
                     "Names": ["MuMu_A", "MuMu_B", "MuMu_C", "MuMu_D", "MuMu_E", "MuMu_F", "MuMu_G", "MuMu_H",
                               "ElMu_A", "ElMu_B", "ElMu_C", "ElMu_D", "ElMu_E", "ElMu_F", "ElMu_G", "ElMu_H",
                               "ElEl_A", "ElEl_B", "ElEl_C", "ElEl_D", "ElEl_E", "ElEl_F", "ElEl_G", "ElEl_H",
                               "El_A", "El_B", "El_C", "El_D", "El_E", "El_F",
                               "Mu_A", "Mu_B", "Mu_C", "Mu_D", "Mu_E", "Mu_F",
                               "ElMu", "ElEl", "El", "Mu",],
                     "Style": "Marker",
                    },
            "QCD": {"Color": ROOT.kPink,
                    "Names": ["QCD_HT200", "QCD_HT300", "QCD_HT500", "QCD_HT700", 
                              "QCD_HT1000", "QCD_HT1500", "QCD_HT2000"],
                    "Style": "Fill",
                   },
        },
        "Supercategories": {
            "Signal+Background": {"Names": ["tttt", "ttbar", "ttH", "ttVJets", "ttultrarare", "DY", "QCD"],
                                  "Stack": True
                                 },
            "Data": {"Names": ["Data"],
                     "Stack": False
                    },
        }
    },
    "DefaultCanvas":{
        "Type": "DefaultCanvas",
        "Name": "DefaultCanvasName",
        "Plots": None,
        "XPixels": 800, 
        "YPixels": 800,
        "DoRatio": False,
        "Unblind": False,
    },
    "Plot_nJet4_Muon_pfRelIso03_chg":{
        "Type": "PlotConfig",
        "Name": "DefaultPlotName",
        "Xaxis": None,
        "Yaxis": None,
        "Rebin": None,
        "Style": {"MC": "Fill",
                  "Data": "Marker",
                 },
        "Files": "nJet4_Muon_pfRelIso03_chg.root",
    },
    "Canvas_Muon_pfRelIso03_chg":{
        "Type": "CanvasConfig",
        "Name": "Muon pfRelIso03 (charged component)",
        "Plots": ["Plot_nJet4_Muon_pfRelIso03_chg"],
        "Legend": "DefaultLegend",
        "XPixels": 800, 
        "YPixels": 800,
        "DoRatio": True,
        "Unblind": False,
    },
}

In [18]:
#pprint.pprint(argsDOTplotJSON)

In [52]:
def createRatio(h1, h2, Cache=None, ratioTitle="input 0 vs input 1", ratioColor = ROOT.kBlack, yMin = 0, yMax = 1.1):
    #h3 = h1.Clone("rat_{}_{}".format(h1.GetName(), ratioTitle.replace(" ", "_")))
    #h3 = h1.Clone("rat_{}".format(h1.GetName()))
    h3 = h1.Clone("ratio_{}__{}".format( (h1.GetName()).replace("h_",""), (h2.GetName()).replace("h_","") ))
    h3.SetLineColor(ratioColor)
    h3.SetMarkerStyle(21)
    # h3.SetTitle("")
    # Set up plot for markers and errors according to ROOT example, but SetStats(0) might be too minimal sometimes
    h3.Sumw2()
    h3.SetStats(0)
    h3.Divide(h2)
    h3.SetMinimum(yMin)
    h3.SetMaximum(yMax)

    # Adjust y-axis settings
    y = h3.GetYaxis()
    y.SetTitle(ratioTitle)
    y.SetNdivisions(505)
    y.SetTitleSize(20)
    y.SetTitleFont(43)
    y.SetTitleOffset(1.55)
    y.SetLabelFont(43)
    y.SetLabelSize(15)

    # Adjust x-axis settings
    x = h3.GetXaxis()
    x.SetTitleSize(20)
    x.SetTitleFont(43)
    x.SetTitleOffset(4.0)
    x.SetLabelFont(43)
    x.SetLabelSize(15)

    if Cache == None:
        Cache = {}
    #These keys will not be sufficient for multiple ratios to be plotted together, #FIXME
    Cache["ratio_hist"] = h3
    Cache["ratio_Xaxis"] = x
    Cache["ratio_Yaxis"] = y
    return Cache


def createCanvasPads(canvasTitle, Cache=None, doRatio=True, setXGrid=False, setYGrid=False,
                     nXPads=1, topFraction=0.7, xPixels=800, yPixels=800):
    """Create canvas with two pads vertically for each of doLin and doLog if they are true"""
    #Divide implicitely creates subpads. This function uses more explicit methods to do the same with varying pad sizes
    c = ROOT.TCanvas(canvasTitle, canvasTitle, xPixels, yPixels)
    # Upper histogram plot is pad1
    upperPads = []
    lowerPads = []
    xEdgesLow = [z/float(nXPads) for z in xrange(nXPads)]
    xEdgesHigh = [(z+1)/float(nXPads) for z in xrange(nXPads)]
    if doRatio:
        yDivision = 1-topFraction
    else:
        yDivision = 0


    for z in xrange(nXPads):
        c.cd()  # returns to main canvas before defining another pad, to not create sub-subpad
        padU = ROOT.TPad("{}_{}".format(canvasTitle,z), "{}_{}".format(canvasTitle,z), 
                        xEdgesLow[z], 1-topFraction, xEdgesHigh[z], 1.0) #xmin ymin xmax ymax as fraction
        if doRatio:
            padU.SetBottomMargin(0)  # joins upper and lower plot
        if setXGrid:
            padU.SetGridx()
        if setYGrid:
            padU.SetGridy()
        padU.Draw()
        if doRatio:
            # Lower ratio plot is pad2
            padL = ROOT.TPad("ratio_{}_{}".format(canvasTitle,z), "ratio_{}_{}".format(canvasTitle,z), 
                             xEdgesLow[z], 0.05, xEdgesHigh[z], 1-topFraction) #xmin ymin xmax ymax as fraction
            padL.SetTopMargin(0)  # joins upper and lower plot
            padL.SetBottomMargin(0.2)
            if setXGrid:
                padL.SetGridx()
            if setYGrid:
                padL.SetGridy()
            padL.Draw()
        upperPads.append(padU)
        lowerPads.append(padL)
    if Cache == None:
        Cache = {}
    Cache["canvas"] = c
    Cache["canvas/upperPads"] = upperPads
    Cache["canvas/lowerPads"] = lowerPads
    return Cache

def getLabelAndHeader(Cache=None, label="#bf{CMS Internal}", header="#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format("PLACEHOLDER")):
    # Add header
    cms_label = ROOT.TLatex()
    cms_label.SetTextSize(0.04)
    cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Internal}")
    header = ROOT.TLatex()
    header.SetTextSize(0.03)
    header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
    if Cache == None:
        Cache = {}
    Cache["cms_label"] = cms_label
    Cache["cms_header"] = cms_header
    return Cache

def addHists(inputHists, name, scaleArray = None):
    retHist = None
    for hn, hist in enumerate(inputHists):
        if hn == 0:
            retHist = hist.Clone("{}".format(name))
            if scaleArray != None and len(scaleArray) == len(inputHists):
                retHist.Scale(scaleArray[hn])
        else:
            if scaleArray != None and len(scaleArray) == len(inputHists):
                retHist.Add(hist, scaleArray[hn])
            else:
                retHist.Add(hist)
    return retHist

def makeCategoryHists(histFile, legendConfig, rootNamePrefix, systematic=None, debug=False):
    """Function tailored to using a legendConfig to create histograms of added """
    #Take the legendConfig, which includes coloring and style information in addition to 
    #The Categories group contains the name of each category along with its color, style, and list of histogram (base) names
    #Particular details like title, axes, etc. are left to the higher level function to change.
    if type(legendConfig) != dict or "Categories" not in legendConfig.keys():
        raise ValueError("legendConfig passed to makeCategoryHists contains no 'Categories' key")
    histKeys = set([hist.GetName() for hist in histFile.GetListOfKeys()])
    #Create dictionary of histograms to be returned by the function
    retHists = {}
    #Create a baseName using the passed rootPrefixName and systematic, if non-None. Will be combined with category information
    #when making each added histogram's name
    baseName = None
    if systematic == None:
        baseName = rootNamePrefix + "*"
    else:
        baseName = rootNamePrefix + "_" + systematic + "*"
    #Cycle through the config's categories
    for cat, config in legendConfig["Categories"].items():
        histoList = []
        addHistoName = baseName + cat
        scaleArray = config.get("ScaleArray", None)
        for nn, hname in enumerate(config["Names"]):
            if debug: print("Creating addHistoName {}".format(addHistoName))
            #Skip plots that contain neither the systematic requested nor the nominal histogram
            if systematic != None:
                #guard against config file that has more histos than are in the file itself
                if hname + "_" + systematic not in histKeys and hname not in histKeys:
                    print("for {} and rootNamePrefix {}, makeCombinationHists failed to find a histogram (systematic or nominal) corresponding to {}"\
                          .format(histFile.GetName(), rootNamePrefix, hname))
                    continue
                else:
                    #Append the histo to a list which will be added using a dedicated function
                    histoList.append(histFile.Get(hname + "_" + systematic))
            elif systematic == None:
                #guard against config file that has more histos than are in the file itself
                if hname not in histKeys:
                    print("for {} and rootNamePrefix {}, makeCombinationHists failed to find a histogram (nominal) corresponding to {}"\
                          .format(histFile.GetName(), rootNamePrefix, hname))
                    continue
                else:
                    #Append the histo to a list which will be added using a dedicated function
                    histoList.append(histFile.Get(hname))
            

        #Make the new histogram with addHistoName, optionally with per-histogram scaling factors
        #print(histoList)
        #print(addHistoName)
        
        if len(histoList) == 0:
            continue
        else:
            retHists[cat] = addHists(histoList, addHistoName, scaleArray = scaleArray)
            #print(retHists)
            #Modify the histogram with style and color information, where appropriate
            if config["Style"] == "Fill":
                retHists[cat].SetFillColor(config["Color"])
            elif config["Style"] == "FillAlpha":
                retHists[cat].SetFillColorAlpha(config["Color"], config.get("Alpha", 0.5))
            elif config["Style"] == "Line":     
                retHists[cat].SetLineColor(config["Color"])
            elif config["Style"] == "Marker":    
                retHists[cat].SetMarkerStyle(0)
                retHists[cat].SetMarkerSize(1.0)
                retHists[cat].SetMarkerColor(config["Color"])
            else:
                pass
    return retHists

def makeSuperCategories(histFile, legendConfig, rootName, systematic=None, orderByIntegral=True, debug=False):
    """histFile is an open ROOT file containing histograms without subdirectories, legendConfig contains 'Categories'
    with key: value pairs of sample categories (like ttbar or Drell-Yan) and corresponding list of histogram sample names
    (like tt_SL, tt_SL-GF, tt_DL, etc.) that are subcomponents of the sample)
    The 'SuperCategories' in the configuration contains key: value pairs where the list then references the 'Categories'
    to be stacked together.
    However, category names should not include any _$SYSTEMATIC postfix; instead, it will be assumed to be appended to each histogram's category name,
    so the histograms made for each category will include the _$SYSTEMATIC variation if present, and the nominal if not."""
    retDict = {}
    coord = legendConfig["Coordinates"]
    leg = ROOT.TLegend(coord[0], coord[1], coord[2], coord[3])
    #leg.SetBorderSize(0)
    #Try to make a ~square legend by creation of columns
    nColumns = math.floor(math.sqrt(len(legendConfig.get("Categories"))))
    leg.SetNColumns(2)
    leg.SetTextSize(0.03)
    
        #leg.AddEntry(leg_hists[color], samplecategory, "F")
        #leg.AddEntry(leg_hists[color], samplecategory, "P")
    #Create dictionary to return one level up, calling makeCategoryHists to combine subsamples together 
    #and do color, style configuration for them
    retDict["Categories/hists"] = makeCategoryHists(histFile, legendConfig, rootNamePrefix=rootName,
                                                    systematic=systematic, debug=debug)
    #Create an ordered list of tuples using either the integral of each category histogram or just the name (for consistency)
    orderingList = []
    for cat_name, cat_hist in retDict["Categories/hists"].items():
        orderingList.append((cat_hist.GetSumOfWeights(), cat_name, cat_hist, ))
    if orderByIntegral:
        orderingList.sort(key=lambda j: j[0], reverse=True)
    else:
        orderingList.sort(key=lambda j: j[1], reverse=False)
    #Create dictionary of supercategory items
    retDict["Supercategories"] = {}
    retDict["Supercategories/testCanvases"] = {}
    retDict["Supercategories/stats"] = {}
    #test only
    testCanvas = {}
    print("The orderingList at this point...")
    print(orderingList)
    for super_cat_name, super_cat_list in legendConfig["Supercategories"].items():
        #seperate out the orderedLists into sublists which can be combined differently for stacked and unstacked types
        #superCategories["{}/list".format(super_cat_name)] = [tup for tup in orderingList if orderingList[1] in super_cat_list["Names"]]
        print("the list of names to check in the supercategory: {}".format(super_cat_list["Names"]))
        
        tmpList = [tup for tup in orderingList if tup[1] in super_cat_list["Names"]]
        retDict["Supercategories"][super_cat_name] = ROOT.THStack("s_{}*{}".format(rootName, super_cat_name), "")
        print("The tuple list...")
        print(tmpList)
        for tup in tmpList:
            legendCode = legendConfig["Categories"][tup[1]]["Style"]
            if legendCode == "Fill" or legendCode == "FillAlpha":
                legendCode = "F"
            elif legendCode == "Line":
                legendCode = "L"
            else:
                #Assume Marker style
                legendCode = "P"
            leg.AddEntry(tup[2], tup[1], legendCode)
            retDict["Supercategories"][super_cat_name].Add(tup[2])
            
        #test
        testCanvas[super_cat_name] = ROOT.TCanvas("c_" + rootName + "*" + super_cat_name)
        #testCanvas[super_cat_name].cd()
        #retDict["Supercategories"][super_cat_name].Draw()
        #testCanvas[super_cat_name].Draw()
        
        retDict["Supercategories/testCanvases"][super_cat_name] = testCanvas[super_cat_name]
    return retDict

def makeStack_Prototype(histFile, histList=None, legendConfig=None, rootName=None, systematic=None, orderByIntegral=True):
    """histFile is an open ROOT file containing histograms without subdirectories, histList is a python list of the name of histograms to be stacked together,
    not including any _$SYSTEMATIC postfix; this latter part will be assumed to be appended to each histogram, and when the systematic option is used,
    this function will stack histograms that have _$SYSTEMATIC postfix; if such a histogram is not present, the nominal histogram without a _$SYSTEMATIC will
    be used.
    
    legendConfig should be a dictionary containing subgrouping, color, and style information about the samples to be loaded"""
    if histList == None and legendConfig == None:
        raise ValueError("makeStack has received no histList and no legendConfig to create a stack from...")
    elif histList == None:
        pass 
    #Check that the desired histograms is actually present in the root file, separating into category where systematic variation is present and fallback nominal
    hists_systematic_set = set([])
    if systematic != None:
        hists_systematic_set = set([inclusion for inclusion in histList for hist in histFile.GetListOfKeys() if "{}_{}".format(inclusion, systematic) == hist.GetName()])
    hists_nominal_set = set([inclusion for inclusion in histList for hist in histFile.GetListOfKeys() if inclusion == hist.GetName()])
    hists_nominal_set = hist_nominal - hists_systematic
    hists_systematic = ["{}_{}".format(inclusion, systematic) for inclusion in hists_systematic_set]
    hists_nominal = [inclusion for inclusion in hists_nominal_set]
    print(hists_systematic)
    print(hists_nominal)
    
    blarg = {}
    blarg["Canvas_Muon_pfRelIso03_chg"] = {
        "Type": "CanvasConfig",
        "Name": "Muon pfRelIso03 (charged component)",
        "Plots": ["Plot_nJet4_Muon_pfRelIso03_chg"],
        "Legend": "DefaultLegend",
        "XPixels": 800, 
        "YPixels": 800,
        "DoRatio": False,
        "Unblind": False,
    }
    blarg["Plot_nJet4_Muon_pfRelIso03_chg"] = {
        "Type": "PlotConfig",
        "Name": "DefaultPlotName",
        "Xaxis": None,
        "Yaxis": None,
        "Rebin": None,
        "Style": {"MC": "Fill",
                  "Data": "Marker",
                 },
        "Files": "nJet4_Muon_pfRelIso03_chg.root",
    },
    
def loopPlotJSON(inputJSON, Cache=None, directory="."):
    """Loop through a JSON encoded plotcard to draw plots based on root files containing histograms.
    Must pass a cache (python dictionary) to the function to prevent python from garbage collecting everything."""
    
    #Parse the config file into non-default legends, canvases, and plots; make a separate dictionary for the default legend, plot, and canvas which 
    # are the fallback for any options
    legends = dict([(i, j) for i, j in inputJSON.items() if j.get("Type") == "LegendConfig"])
    canvases = dict([(i, j) for i, j in inputJSON.items() if j.get("Type") == "CanvasConfig"])
    plots = dict([(i, j) for i, j in inputJSON.items() if j.get("Type") == "PlotConfig"])
    defaults = dict([(i, j) for i, j in inputJSON.items() if j.get("Type") in ["DefaultPlot", "DefaultCanvas", "DefaultLegend"]])
    
    pprint.pprint(plots)
    #Cache everything, we don't want python garbage collecting our objects before we're done using them. 
    if Cache == None:
        Cache = {}
    leg_num = 0
    plt_num = 0
    can_num = 0
    #Loop through the canvases to create. These are the head configurations for making our plots. The Cache should be filled with CanvasConfigs (unique), 
    #because PlotConfigs may be reused!
    #Each CanvasConfig should point to a LegendConfig, which in turn configures how to organize the individual histograms for stacks and grouping
    #Each CanvasConfig also must point to a number of PlotConfigs, where a mountain range canvas of 3 categories would point to 3 PlotConfigs.
    #The PlotConfig itself points to the root file which contains all the sample/data subcategories for that individual histogram 
    # (for example, ttbar_SL and tttt for the leading Muon_pt). 
    print("Looping through canvases")
    print(canvases)
    for can_name, can_dict in canvases.items():
        can_num += 1
        CanCache = {} #shorter access to this canvas dictionary
        Cache[can_name] = CanCache
        
        #Acquire the details of this canvas, including the list of (sub)plots, the number of pixels, whether to include a ratio, etc.
        CanCache["subplots"] = can_dict.get("Plots")
        nXPads = len(subplots)
        xPixels=can_dict.get("XPixels", defaults["DefaultCanvas"].get("XPixels"))
        yPixels=can_dict.get("YPixels", defaults["DefaultCanvas"].get("YPixels"))
        doRatio=can_dict.get("DoRatio", defaults["DefaultCanvas"].get("DoRatio"))
        
        #Load the LegendConfig which denotes which samples to use, colors to assign, etc.
        
        #Call createCanvasPads with our Can(vas)Cache, allowing us to toss the returned dictionary.
        _ = createCanvasPads(can_name, CanCache, doRatio = can_dict.get("DoRatio"), setXGrid=False, setYGrid=False,
                     nXPads=nXPads, topFraction=0.7, xPixels=xPixels, yPixels=yPixels)
        CanCache["subplots/files"] = []
        CanCache["subplots/channels"] = []
        CanCache["subplots/histograms"] = []
        CanCache["subplots/stats"] = []
        for pn, subplot_name in enumerate(CanCache["subplots"]):
            subplot_dict = can_dict["{}".format(subplot_name)]
            CanCache["subplots/files"].append(ROOT.TFile.Open("{}".format(subplot_dict["Files"])))
            
            
    for plt_name, plt_dict in plots.items():
        plt_num += 1
        Cache[plt_name] = {}
        try:
            #Form the path to the histogram file using the directory and the file name specified in the plotcard
            print("Opening file... {}".format(plt_dict.get("Files")))
            f = ROOT.TFile.Open(directory + "/" + plt_dict.get("Files"))
            Cache[plt_name]["File"] = f
            for k in f.GetListOfKeys():
                hist_name = k.GetName()
                Cache[plt_name][hist_name] = f.Get(hist_name)
                Cache[plt_name]["c_{}".format(hist_name)] = ROOT.TCanvas("c_{}_{}".format(plt_name, hist_name), "", 800, 600)
                Cache[plt_name]["c_{}".format(hist_name)].cd()
                Cache[plt_name][hist_name].SetLineColor(ROOT.kRed)
                Cache[plt_name][hist_name].Draw("HIST")
                Cache[plt_name]["c_{}".format(hist_name)].Draw()
        except:
            print("Failed in execution of loopPlotJSON")

def loopPlotJSON_Prototype(inputJSON, Cache=None, directory="."):
    """Loop through a JSON encoded plotcard to draw plots based on root files containing histograms.
    Must pass a cache (python dictionary) to the function to prevent python from garbage collecting everything."""
    legends = [(i, j) for i, j in inputJSON.items() if j.get("Type") == "LegendConfig"]
    canvases = [(i, j) for i, j in inputJSON.items() if j.get("Type") == "CanvasConfig"]
    plots = [(i, j) for i, j in inputJSON.items() if j.get("Type") == "PlotConfig"]
    
    pprint.pprint(plots)
    if Cache == None:
        Cache = {}
    plt_num = 0
    for plt_name, plt_dict in plots:
        plt_num += 1
        Cache[plt_name] = {}
        try:
            #Form the path to the histogram file using the directory and the file name specified in the plotcard
            print("Opening file... {}".format(plt_dict.get("Files")))
            f = ROOT.TFile.Open(directory + "/" + plt_dict.get("Files"))
            Cache[plt_name]["File"] = f
            for k in f.GetListOfKeys():
                hist_name = k.GetName()
                Cache[plt_name][hist_name] = f.Get(hist_name)
                Cache[plt_name]["c_{}".format(hist_name)] = ROOT.TCanvas("c_{}_{}".format(plt_name, hist_name), "", 800, 600)
                Cache[plt_name]["c_{}".format(hist_name)].cd()
                Cache[plt_name][hist_name].SetLineColor(ROOT.kRed)
                Cache[plt_name][hist_name].Draw("HIST")
                Cache[plt_name]["c_{}".format(hist_name)].Draw()
        except:
            print("Failed in execution of loopPlotJSON")
            
            

In [53]:
test = ROOT.TFile.Open("test_20200123/nJet4_Muon_pfRelIso03_chg.root")
testDict = makeSuperCategories(test, argsDOTplotJSON["DefaultLegend"], "aTestName", systematic=None, orderByIntegral=True, debug=False)

for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to ttH
for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to ttWJets
for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to ttZJets
for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to DYJets_DL
for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to MuMu_A
for test_20200123/nJet4_Muon_pfRelIso03_chg.root and rootNamePrefix aTestName, makeCombinationHists failed to find a histogram (nominal) corresponding to MuMu_B
for test_20200123/nJet4_Muon_pfR

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_aTestName*Data
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_aTestName*Signal+Background


In [55]:
%jsroot on
print(testDict)
c = ROOT.TCanvas("cccccc", "", 800, 600)
c.cd()
l = []
l.append(testDict['Categories/hists']['singletop'].GetSumOfWeights())
print(l)
print(type(testDict['Supercategories']['Signal+Background']))
testDict['Supercategories']['Signal+Background'].Draw("HIST")
testDict['Categories/hists']['singletop'].Draw("PE! SAMES")
c.Draw()

{'Supercategories/testCanvases': {'Data': <ROOT.TCanvas object ("c_aTestName*Data") at 0x3f217f0>, 'Signal+Background': <ROOT.TCanvas object ("c_aTestName*Signal+Background") at 0x3ff0120>}, 'Categories/hists': {'singletop': <ROOT.TH1D object ("aTestName*singletop") at 0x3ed7e40>, 'ttbar': <ROOT.TH1D object ("aTestName*ttbar") at 0x3f72800>, 'tttt': <ROOT.TH1D object ("aTestName*tttt") at 0x3b71510>}, 'Supercategories/stats': {}, 'Supercategories': {'Data': <ROOT.THStack object ("s_aTestName*Data") at 0x3db2a50>, 'Signal+Background': <ROOT.THStack object ("s_aTestName*Signal+Background") at 0x3f21b80>}}
[76.52655407044864]
<class 'ROOT.THStack'>


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cccccc


In [48]:
%jsroot on
mycache = {}
loopPlotJSON(argsDOTplotJSON, mycache, "test_20200123")

{'Plot_nJet4_Muon_pfRelIso03_chg': {'Files': 'nJet4_Muon_pfRelIso03_chg.root',
                                    'Name': 'DefaultPlotName',
                                    'Rebin': None,
                                    'Style': {'Data': 'Marker', 'MC': 'Fill'},
                                    'Type': 'PlotConfig',
                                    'Xaxis': None,
                                    'Yaxis': None}}
Looping through canvases
{'Canvas_Muon_pfRelIso03_chg': {'XPixels': 800, 'Name': 'Muon pfRelIso03 (charged component)', 'DoRatio': True, 'Plots': ['Plot_nJet4_Muon_pfRelIso03_chg'], 'Type': 'CanvasConfig', 'Legend': 'DefaultLegend', 'Unblind': False, 'YPixels': 800}}
{'lowerPads': [<ROOT.TPad object ("ratio_Canvas_Muon_pfRelIso03_chg_0") at 0x29b0960>], 'canvas': <ROOT.TCanvas object ("Canvas_Muon_pfRelIso03_chg") at 0x2e5d970>, 'upperPads': [<ROOT.TPad object ("Canvas_Muon_pfRelIso03_chg_0") at 0x29ac400>]}
{'lowerPads': [<ROOT.TPad object ("ratio_Canvas_Muon_

In [91]:
f = ROOT.TFile.Open("test_20200123" + "/" + "nJet4_Muon_pfRelIso03_chg.root")
f.ls()
c = ROOT.TCanvas("c")
c.Divide(1,3)
for kn, k in enumerate(f.GetListOfKeys()):  
    c.cd(kn + 1)
    h = f.Get(k.GetName())
    h.Draw()
c.Draw()

TFile**		test_20200123/nJet4_Muon_pfRelIso03_chg.root	
 TFile*		test_20200123/nJet4_Muon_pfRelIso03_chg.root	
  KEY: TH1D	tt_DL;1	
  KEY: TH1D	tttt;1	
  KEY: TH1D	ST_tW;1	


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [3]:
masterstart = time.time() #.clock() gives cpu time, not what I want for this measure (especially multicore)
with open("2017_booker.py", "r") as f:
    for line in f:
        #print(line, end="")
        continue

In [10]:
#FIXME: Need filter efficiency calculated for single lepton generator filtered sample. First approximation will be from MCCM (0.15) but as seen before, it's not ideal. 
#May need to recalculate using genWeight/sumWeights instead of sign(genWeight)/(nPositiveEvents - nNegativeEvents), confirm if there's any difference.
lumi = {"2017": 41.53,
        "2018": 1}
era = "2017"
leg_dict = {"tttt": ROOT.kAzure-2,
            "ttbar": ROOT.kRed,
            "singletop": ROOT.kYellow,
            "ttH": ROOT.kMagenta,
            "ttVJets": ROOT.kViolet,
            "ttultrarare": ROOT.kGreen,
            "DY": ROOT.kCyan,
            "Data": ROOT.kBlack,
            "QCD": ROOT.kPink,
           }
print(leg_dict)
microbookerV2 = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttt-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttt-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt-2_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/tttt/$SYSTEMATIC",
    },
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-2_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-3_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-4_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-5_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-6_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-7_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-8_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-9_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-10_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-11_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-12_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-13_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-14_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_DL/$SYSTEMATIC",
    },
    "tt_SL":{
        "era": "2017",
        "isData": False,
        "nEvents": 20122010,
        "nEventsPositive": 20040607,
        "nEventsNegative": 81403,
        "sumWeights": 6052480345.748356,
        "sumWeights2": 1850350248120.376221,
        "isSignal": False,
        "crossSection": 366.2073,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_SL/$SYSTEMATIC",
    },

    "ST_tW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7945242,
        "nEventsPositive": 7914815,
        "nEventsNegative": 30427,
        "sumWeights": 277241050.840222,
        "sumWeights2": 9823995766.508368,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tW_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/ST_tW/$SYSTEMATIC",
    },
    "ST_tbarW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7745276,
        "nEventsPositive": 7715654,
        "nEventsNegative": 30427,
        "sumWeights": 270762750.172525,
        "sumWeights2": 9611964941.797768,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tbarW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tbarW_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/ST_tbarW/$SYSTEMATIC",
    },
    "DYJets_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 49125561,
        "nEventsPositive": 49103859,
        "nEventsNegative": 21702,
        "sumWeights": 49082157.000000,
        "sumWeights2": 49125561.000000,
        "isSignal": False,
        "crossSection": 5075.6,
        "color": leg_dict["DY"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-1_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-2_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-3_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-4_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-5_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-6_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-7_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/DYJets_DL/$SYSTEMATIC",
    },
}
tt_data_V2 = {
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-2_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-3_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-4_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-5_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-6_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-7_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-8_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-9_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-10_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-11_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-12_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-13_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-14_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_DL/$SYSTEMATIC",
    },
    "tt_SL":{
        "era": "2017",
        "isData": False,
        "nEvents": 20122010,
        "nEventsPositive": 20040607,
        "nEventsNegative": 81403,
        "sumWeights": 6052480345.748356,
        "sumWeights2": 1850350248120.376221,
        "isSignal": False,
        "crossSection": 366.2073,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_SL/$SYSTEMATIC",
    },
    "ElMu":{
        "era": "2017",
        "subera": "BCDEF",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 4453465 + 15595214 + 9164365 + 19043421 + 25776363,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_*_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_*_2017.root",],
        "destination": "/$HIST_CLASS/$HIST/ElMu/$SYSTEMATIC",
    },    
}
bookerV2 = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttt-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttt-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt-2_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/tttt/$SYSTEMATIC",
    },
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-2_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-3_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-4_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-5_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-6_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-7_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-8_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-9_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-10_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-11_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-12_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-13_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-NOM-14_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_DL/$SYSTEMATIC",
    },
    "tt_SL":{
        "era": "2017",
        "isData": False,
        "nEvents": 20122010,
        "nEventsPositive": 20040607,
        "nEventsNegative": 81403,
        "sumWeights": 6052480345.748356,
        "sumWeights2": 1850350248120.376221,
        "isSignal": False,
        "crossSection": 366.2073,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-NOM_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_SL/$SYSTEMATIC",
    },

    "ST_tW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7945242,
        "nEventsPositive": 7914815,
        "nEventsNegative": 30427,
        "sumWeights": 277241050.840222,
        "sumWeights2": 9823995766.508368,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tW_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/ST_tW/$SYSTEMATIC",
    },
    "ST_tbarW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7745276,
        "nEventsPositive": 7715654,
        "nEventsNegative": 30427,
        "sumWeights": 270762750.172525,
        "sumWeights2": 9611964941.797768,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tbarW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ST_tbarW_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/ST_tbarW/$SYSTEMATIC",
    },
    "DYJets_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 49125561,
        "nEventsPositive": 49103859,
        "nEventsNegative": 21702,
        "sumWeights": 49082157.000000,
        "sumWeights2": 49125561.000000,
        "isSignal": False,
        "crossSection": 5075.6,
        "color": leg_dict["DY"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-1_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-2_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-3_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-4_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-5_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-6_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/DYJets_DL-7_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/DYJets_DL/$SYSTEMATIC",
    },
    "ttH":{
        "era": "2017",
        "isData": False,
        "nEvents": 8000000,
        "nEventsPositive": 7916867,
        "nEventsNegative": 83133,
        "sumWeights": 4216319.315884,
        "sumWeights2": 2317497.816608,
        "isSignal": False,
        "crossSection": 0.2934,
        "color": leg_dict["ttH"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttH_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttH_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttH/$SYSTEMATIC",
    },
    "ttWJets":{
        "era": "2017",
        "isData": False,
        "nEvents": 9425384,
        "nEventsPositive": 9404856,
        "nEventsNegative": 20528,
        "sumWeights": 9384328.000000,
        "sumWeights2": 9425384.000000,
        "isSignal": False,
        "crossSection": 0.611,
        "color": leg_dict["ttVJets"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWJets_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWJets_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttWJets/$SYSTEMATIC",
    },
    "ttZJets":{
        "era": "2017",
        "isData": False,
        "nEvents": 8536618,
        "nEventsPositive": 8527846,
        "nEventsNegative": 8772,
        "sumWeights": 8519074.000000,
        "sumWeights2": 8536618.000000,
        "isSignal": False,
        "crossSection": 0.783,
        "color": leg_dict["ttVJets"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZJets_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZJets_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttZJets/$SYSTEMATIC",
    },
    "ttWH":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199491,
        "nEventsNegative": 509,
        "sumWeights": 198839.680865,
        "sumWeights2": 199704.039588,
        "isSignal": False,
        "crossSection": 0.001572,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWH_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWH_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttWH/$SYSTEMATIC",
    },
    "ttWW":{
        "era": "2017",
        "isData": False,
        "nEvents": 962000,
        "nEventsPositive": 962000,
        "nEventsNegative": 0,
        "sumWeights": 962000.000000,
        "sumWeights2": 962000.000000,
        "isSignal": False,
        "crossSection": 0.007882,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWW_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttWW/$SYSTEMATIC",
    },
    "ttWZ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199379,
        "nEventsNegative": 621,
        "sumWeights": 198625.183551,
        "sumWeights2": 199708.972601,
        "isSignal": False,
        "crossSection": 0.002974,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWZ_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttWZ_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttWZ/$SYSTEMATIC",
    },
    "ttZZ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199686,
        "nEventsNegative": 314,
        "sumWeights": 199286.628891,
        "sumWeights2": 199816.306332,
        "isSignal": False,
        "crossSection": 0.001572,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZZ_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZZ_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttZZ/$SYSTEMATIC",
    },
    "ttZH":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199643,
        "nEventsNegative": 357,
        "sumWeights": 199192.234990,
        "sumWeights2": 199794.753976,
        "isSignal": False,
        "crossSection": 0.01253,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZH_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttZH_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttZH/$SYSTEMATIC",
    },
    "ttHH":{
        "era": "2017",
        "isData": False,
        "nEvents": 194817,
        "nEventsPositive": 194516,
        "nEventsNegative": 301,
        "sumWeights": 194116.909912,
        "sumWeights2": 194611.090542,
        "isSignal": False,
        "crossSection": 0.0007408,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttHH_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ttHH_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ttHH/$SYSTEMATIC",
    },
    "tttJ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199273,
        "nEventsNegative": 727,
        "sumWeights": 198394.878491,
        "sumWeights2": 199663.384954,
        "isSignal": False,
        "crossSection": 0.0004741,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttJ_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttJ_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tttJ/$SYSTEMATIC",
    },
    "ElMu":{
        "era": "2017",
        "subera": "BCDEF",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 4453465 + 15595214 + 9164365 + 19043421 + 25776363,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_B_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_C_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_D_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_E_2017_v2.root",
                        "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/data_ElMu_F_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/ElMu/NOMINAL",
    },
}
cutoutV2_ToBeFixed = {
    "QCD_HT200":{
        "era": "2017",
        "isData": False,
        "nEvents": 59200263,
        "nEventsPositive": 59166789,
        "nEventsNegative": 32544,
        "sumWeights": 59133315.000000,
        "sumWeights2": 59200263.000000,
        "isSignal": False,
        "crossSection": 1712000.0,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT200_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT200_2017_v2.root",],
    },
    "QCD_HT300":{
        "era": "2017",
        "isData": False,
        "nEvents": 59569132,
        "nEventsPositive": 59514373,
        "nEventsNegative": 54759,
        "sumWeights": 59459614.000000,
        "sumWeights2": 59569132.000000,
        "isSignal": False,
        "crossSection": 347700.0,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT300_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT300_2017_v2.root",],
    },   
    "QCD_HT500":{
        "era": "2017",
        "isData": False,
        "nEvents": 56207744,
        "nEventsPositive": 56124381,
        "nEventsNegative": 83363,
        "sumWeights": 56041018.000000,
        "sumWeights2": 56207744.000000,
        "isSignal": False,
        "crossSection": 32100.0,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT500_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT500_2017_v2.root",],
    },
    "QCD_HT700":{
        "era": "2017",
        "isData": False,
        "nEvents": 46840955,
        "nEventsPositive": 46739970,
        "nEventsNegative": 100985,
        "sumWeights": 46638985.000000,
        "sumWeights2": 46840955.000000,
        "isSignal": False,
        "crossSection": 6831.0,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT700_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT700_2017_v2.root",],
    },
    "QCD_HT1000":{
        "era": "2017",
        "isData": False,
        "nEvents": 16882838,
        "nEventsPositive": 16826800,
        "nEventsNegative": 56038,
        "sumWeights": 16770762.000000,
        "sumWeights2": 16882838.000000,
        "isSignal": False,
        "crossSection": 1207.0,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT1000_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT1000_2017_v2.root",],
    },
    "QCD_HT1500":{
        "era": "2017",
        "isData": False,
        "nEvents": 11634434,
        "nEventsPositive": 11571519,
        "nEventsNegative": 62915,
        "sumWeights": 11508604.000000,
        "sumWeights2": 11634434.000000,
        "isSignal": False,
        "crossSection": 119.9,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT1500_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT1500_2017_v2.root",],
    },
    "QCD_HT2000":{
        "era": "2017",
        "isData": False,
        "nEvents": 5941306,
        "nEventsPositive": 5883436,
        "nEventsNegative": 57870,
        "sumWeights": 5825566.000000,
        "sumWeights2": 5941306.000000,
        "isSignal": False,
        "crossSection": 25.24,
        "color": leg_dict["QCD"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT2000_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/QCD_HT2000_2017_v2.root",],
    },
    "tt_SL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8836856,
        "nEventsPositive": 8794464,
        "nEventsNegative": 42392,
        "sumWeights": 2653328498.476976,
        "sumWeights2": 812201885978.209229,
        "isSignal": False,
        "crossSection": 6,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-GF_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_SL-GF_2017_v2.root"],
        "destination": "/$HIST_CLASS/$HIST/tt_SL-GF/$SYSTEMATIC",
    },
    "tt_DL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8510388,
        "nEventsPositive": 8467543,
        "nEventsNegative": 42845,
        "sumWeights": 612101836.284397,
        "sumWeights2": 44925503249.097206,
        "isSignal": False,
        "crossSection": 1.4705,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-GF-*_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-GF-1_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-GF-2_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-GF-3_2017_v2.root",
                       "root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tt_DL-GF-4_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tt_DL-GF/$SYSTEMATIC",
    },
    "tttW":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199852,
        "nEventsNegative": 148,
        "sumWeights": 199552.187377,
        "sumWeights2": 199697.648421,
        "isSignal": False,
        "crossSection": 0.007882,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttW_2017_v2.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/tttW_2017_v2.root",],
        "destination": "/$HIST_CLASS/$HIST/tttW/$SYSTEMATIC",
    },
    "ElMu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 4453465,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_B_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_B_2017.root",],
        },
    "ElMu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 15595214, 
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_C_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_C_2017.root",],
        },
    "ElMu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 9164365,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_D_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_D_2017.root",],
        },
    "ElMu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 19043421,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_E_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_E_2017.root",],
        },
    "ElMu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 25776363,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_F_2017.root",
        "sourceSPARK": ["root://eoshome-n.cern.ch//eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElMu_F_2017.root",],
        },
    "MuMu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 14501767,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/MuMu_B_2017.root",
        },
    "MuMu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 49636525,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/MuMu_C_2017.root",
        },
    "MuMu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 23075733,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/MuMu_D_2017.root",
        },
    "MuMu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 51589091,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/MuMu_E_2017.root",
        },
    "MuMu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 79756560,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/MuMu_F_2017.root",
        },
    "ElEl_B":{
        "era": "2017",
        "subera": "B",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 58088760,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElEl_B_2017.root",
        },
    "ElEl_C":{
        "era": "2017",
        "subera": "C",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 65181125,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElEl_C_2017.root",
        },
    "ElEl_D":{
        "era": "2017",
        "subera": "D",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 25911432,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElEl_D_2017.root",
        },
    "ElEl_E":{
        "era": "2017",
        "subera": "E",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 56233597,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElEl_E_2017.root",
        },
    "ElEl_F":{
        "era": "2017",
        "subera": "F",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 74307066,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/ElEl_F_2017.root",
        },
    "Mu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "Mu",
        "isData": True,
        "nEvents": 136300266,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/Mu_B_2017.root",
        },
    "Mu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "Mu",
        "isData": True,
        "nEvents": 165652756,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/Mu_C_2017.root",
        },
    "Mu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "Mu",
        "isData": True,
        "nEvents": 70361660,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/Mu_D_2017.root",
        },
    "Mu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "Mu",
        "isData": True,
        "nEvents": 154630534,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/Mu_E_2017.root",
        },
    "Mu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "Mu",
        "isData": True,
        "nEvents": 242135500,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/Mu_F_2017.root",
        },
    "El_B":{
        "era": "2017",
        "subera": "B",
        "channel": "El",
        "isData": True,
        "nEvents": 60537490,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/El_B_2017.root",
        },
    "El_C":{
        "era": "2017",
        "subera": "C",
        "channel": "El",
        "isData": True,
        "nEvents": 136637888,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/El_C_2017.root",
        },
    "El_D":{
        "era": "2017",
        "subera": "D",
        "channel": "El",
        "isData": True,
        "nEvents": 51526710,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/El_D_2017.root",
        },
    "El_E":{
        "era": "2017",
        "subera": "E",
        "channel": "El",
        "isData": True,
        "nEvents": 102121689,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/El_E_2017.root",
        },
    "El_F":{
        "era": "2017",
        "subera": "F",
        "channel": "El",
        "isData": True,
        "nEvents": 128467223,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/FilesV2/El_F_2017.root",
    },
}
ttbooker = {
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL_2017_PUFix.root",
        },
}
ttttbooker = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt_2017_PUFix.root",
        },
}
microbooker = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt_2017_PUFix.root",
        },
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL_2017_PUFix.root",
        },
    "ST_tW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7945242,
        "nEventsPositive": 7914815,
        "nEventsNegative": 30427,
        "sumWeights": 277241050.840222,
        "sumWeights2": 9823995766.508368,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ST_tW_2017_PUFix.root",
        },
}
theOriginal = {
    "tttt_orig":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": False,
        "crossSection": 0.012,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt-orig_2.root",
        },
}
pyrdfbooker = {
    "tt_DL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8510388,
        "nEventsPositive": 8467543,
        "nEventsNegative": 42845,
        "sumWeights": 612101836.284397,
        "sumWeights2": 44925503249.097206,
        "isSignal": False,
        "crossSection": 1.4705,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL-GF-*_2017_PUFix.root",
        },
}
booker = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt_2017_PUFix.root",
        },
    "tt_SL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8836856,
        "nEventsPositive": 8794464,
        "nEventsNegative": 42392,
        "sumWeights": 2653328498.476976,
        "sumWeights2": 812201885978.209229,
        "isSignal": False,
        "crossSection": 6,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_SL-GF_2017_PUFix.root",
        },
    "tt_DL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8510388,
        "nEventsPositive": 8467543,
        "nEventsNegative": 42845,
        "sumWeights": 612101836.284397,
        "sumWeights2": 44925503249.097206,
        "isSignal": False,
        "crossSection": 1.4705,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL-GF-*_2017_PUFix.root",
        },
    "tt_SL":{
        "era": "2017",
        "isData": False,
        "nEvents": 20122010,
        "nEventsPositive": 20040607,
        "nEventsNegative": 81403,
        "sumWeights": 6052480345.748356,
        "sumWeights2": 1850350248120.376221,
        "isSignal": False,
        "crossSection": 366.2073,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_SL_2017_PUFix.root",
        },
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL_2017_PUFix.root",
        },
    "ST_tW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7945242,
        "nEventsPositive": 7914815,
        "nEventsNegative": 30427,
        "sumWeights": 277241050.840222,
        "sumWeights2": 9823995766.508368,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ST_tW_2017_PUFix.root",
        },
    "ST_tbarW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7745276,
        "nEventsPositive": 7715654,
        "nEventsNegative": 30427,
        "sumWeights": 270762750.172525,
        "sumWeights2": 9611964941.797768,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ST_tbarW_2017_PUFix.root",
        },
    "ttH":{
        "era": "2017",
        "isData": False,
        "nEvents": 8000000,
        "nEventsPositive": 7916867,
        "nEventsNegative": 83133,
        "sumWeights": 4216319.315884,
        "sumWeights2": 2317497.816608,
        "isSignal": False,
        "crossSection": 0.2934,
        "color": leg_dict["ttH"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttH_2017_PUFix.root",
        },
    "ttWJets":{
        "era": "2017",
        "isData": False,
        "nEvents": 9425384,
        "nEventsPositive": 9404856,
        "nEventsNegative": 20528,
        "sumWeights": 9384328.000000,
        "sumWeights2": 9425384.000000,
        "isSignal": False,
        "crossSection": 0.611,
        "color": leg_dict["ttVJets"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttWJets_2017_PUFix.root",
        },
    "ttZJets":{
        "era": "2017",
        "isData": False,
        "nEvents": 8536618,
        "nEventsPositive": 8527846,
        "nEventsNegative": 8772,
        "sumWeights": 8519074.000000,
        "sumWeights2": 8536618.000000,
        "isSignal": False,
        "crossSection": 0.783,
        "color": leg_dict["ttVJets"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttZJets_2017_PUFix.root",
        },
    "ttWH":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199491,
        "nEventsNegative": 509,
        "sumWeights": 198839.680865,
        "sumWeights2": 199704.039588,
        "isSignal": False,
        "crossSection": 0.001572,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttWH_2017_PUFix.root",
        },
    "ttWW":{
        "era": "2017",
        "isData": False,
        "nEvents": 962000,
        "nEventsPositive": 962000,
        "nEventsNegative": 0,
        "sumWeights": 962000.000000,
        "sumWeights2": 962000.000000,
        "isSignal": False,
        "crossSection": 0.007882,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttWW_2017_PUFix.root",
        },
    "ttWZ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199379,
        "nEventsNegative": 621,
        "sumWeights": 198625.183551,
        "sumWeights2": 199708.972601,
        "isSignal": False,
        "crossSection": 0.002974,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttWZ_2017_PUFix.root",
        },
    "ttZZ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199686,
        "nEventsNegative": 314,
        "sumWeights": 199286.628891,
        "sumWeights2": 199816.306332,
        "isSignal": False,
        "crossSection": 0.001572,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttZZ_2017_PUFix.root",
        },
    "ttZH":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199643,
        "nEventsNegative": 357,
        "sumWeights": 199192.234990,
        "sumWeights2": 199794.753976,
        "isSignal": False,
        "crossSection": 0.01253,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttZH_2017_PUFix.root",
        },
    "ttHH":{
        "era": "2017",
        "isData": False,
        "nEvents": 194817,
        "nEventsPositive": 194516,
        "nEventsNegative": 301,
        "sumWeights": 194116.909912,
        "sumWeights2": 194611.090542,
        "isSignal": False,
        "crossSection": 0.0007408,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ttHH_2017_PUFix.root",
        },
    "tttW":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199852,
        "nEventsNegative": 148,
        "sumWeights": 199552.187377,
        "sumWeights2": 199697.648421,
        "isSignal": False,
        "crossSection": 0.007882,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttW_2017_PUFix.root",
        },
    "tttJ":{
        "era": "2017",
        "isData": False,
        "nEvents": 200000,
        "nEventsPositive": 199273,
        "nEventsNegative": 727,
        "sumWeights": 198394.878491,
        "sumWeights2": 199663.384954,
        "isSignal": False,
        "crossSection": 0.0004741,
        "color": leg_dict["ttultrarare"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttJ_2017_PUFix.root",
        },
    "DYJets_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 49125561,
        "nEventsPositive": 49103859,
        "nEventsNegative": 21702,
        "sumWeights": 49082157.000000,
        "sumWeights2": 49125561.000000,
        "isSignal": False,
        "crossSection": 5075.6,
        "color": leg_dict["DY"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/DYJets_DL_2017_PUFix.root",
        },
    "ElMu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 4453465,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_B_2017.root",
        },
    "ElMu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 15595214,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_C_2017.root",
        },
    "ElMu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 9164365,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_D_2017.root",
        },
    "ElMu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 19043421,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_E_2017.root",
        },
    "ElMu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 25776363,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_F_2017.root",
        },
}
cutout = {
    "MuMu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 14501767,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_B_2017.root",
        },
    "MuMu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 49636525,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_C_2017.root",
        },
    "MuMu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 23075733,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_D_2017.root",
        },
    "MuMu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 51589091,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_E_2017.root",
        },
    "MuMu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 79756560,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_F_2017.root",
        },
    "ElEl_B":{
        "era": "2017",
        "subera": "B",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 58088760,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_B_2017.root",
        },
    "ElEl_C":{
        "era": "2017",
        "subera": "C",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 65181125,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_C_2017.root",
        },
    "ElEl_D":{
        "era": "2017",
        "subera": "D",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 25911432,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_D_2017.root",
        },
    "ElEl_E":{
        "era": "2017",
        "subera": "E",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 56233597,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_E_2017.root",
        },
    "ElEl_F":{
        "era": "2017",
        "subera": "F",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 74307066,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_F_2017.root",
        },
    "Mu_B":{
        "era": "2017",
        "subera": "B",
        "channel": "Mu",
        "isData": True,
        "nEvents": 136300266,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_B_2017.root",
        },
    "Mu_C":{
        "era": "2017",
        "subera": "C",
        "channel": "Mu",
        "isData": True,
        "nEvents": 165652756,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_C_2017.root",
        },
    "Mu_D":{
        "era": "2017",
        "subera": "D",
        "channel": "Mu",
        "isData": True,
        "nEvents": 70361660,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_D_2017.root",
        },
    "Mu_E":{
        "era": "2017",
        "subera": "E",
        "channel": "Mu",
        "isData": True,
        "nEvents": 154630534,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_E_2017.root",
        },
    "Mu_F":{
        "era": "2017",
        "subera": "F",
        "channel": "Mu",
        "isData": True,
        "nEvents": 242135500,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_F_2017.root",
        },
    "El_B":{
        "era": "2017",
        "subera": "B",
        "channel": "El",
        "isData": True,
        "nEvents": 60537490,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_B_2017.root",
        },
    "El_C":{
        "era": "2017",
        "subera": "C",
        "channel": "El",
        "isData": True,
        "nEvents": 136637888,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_C_2017.root",
        },
    "El_D":{
        "era": "2017",
        "subera": "D",
        "channel": "El",
        "isData": True,
        "nEvents": 51526710,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_D_2017.root",
        },
    "El_E":{
        "era": "2017",
        "subera": "E",
        "channel": "El",
        "isData": True,
        "nEvents": 102121689,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_E_2017.root",
        },
    "El_F":{
        "era": "2017",
        "subera": "F",
        "channel": "El",
        "isData": True,
        "nEvents": 128467223,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_F_2017.root",
        },
    }
minibooker = {
    "tttt":{
        "era": "2017",
        "isData": False,
        "nEvents": 2273928,
        "nEventsPositive": 1561946,
        "nEventsNegative": 711982,
        "sumWeights": 18645.487772,
        "sumWeights2": 1094.209551,
        "isSignal": True,
        "crossSection": 0.012,
        "color": leg_dict["tttt"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tttt_2017_PUFix.root",
        },
    "tt_SL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8836856,
        "nEventsPositive": 8794464,
        "nEventsNegative": 42392,
        "sumWeights": 2653328498.476976,
        "sumWeights2": 812201885978.209229,
        "isSignal": False,
        "crossSection": 6,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_SL-GF_2017_PUFix.root",
        },
    "tt_DL-GF":{
        "era": "2017",
        "isData": False,
        "nEvents": 8510388,
        "nEventsPositive": 8467543,
        "nEventsNegative": 42845,
        "sumWeights": 612101836.284397,
        "sumWeights2": 44925503249.097206,
        "isSignal": False,
        "crossSection": 1.4705,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL-GF-*_2017_PUFix.root",
        },
    "tt_SL":{
        "era": "2017",
        "isData": False,
        "nEvents": 20122010,
        "nEventsPositive": 20040607,
        "nEventsNegative": 81403,
        "sumWeights": 6052480345.748356,
        "sumWeights2": 1850350248120.376221,
        "isSignal": False,
        "crossSection": 366.2073,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_SL_2017_PUFix.root",
        },
    "tt_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 69098644,
        "nEventsPositive": 68818780,
        "nEventsNegative": 279864,
        "sumWeights": 4980769113.241218,
        "sumWeights2": 364913493679.955078,
        "isSignal": False,
        "crossSection": 89.0482,
        "color": leg_dict["ttbar"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/tt_DL_2017_PUFix.root",
        },
    "ST_tW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7945242,
        "nEventsPositive": 7914815,
        "nEventsNegative": 30427,
        "sumWeights": 277241050.840222,
        "sumWeights2": 9823995766.508368,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ST_tW_2017_PUFix.root",
        },
    "ST_tbarW":{
        "era": "2017",
        "isData": False,
        "nEvents": 7745276,
        "nEventsPositive": 7715654,
        "nEventsNegative": 30427,
        "sumWeights": 270762750.172525,
        "sumWeights2": 9611964941.797768,
        "isSignal": False,
        "crossSection": 35.8,
        "color": leg_dict["singletop"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ST_tbarW_2017_PUFix.root",
        },

    "DYJets_DL":{
        "era": "2017",
        "isData": False,
        "nEvents": 49125561,
        "nEventsPositive": 49103859,
        "nEventsNegative": 21702,
        "sumWeights": 49082157.000000,
        "sumWeights2": 49125561.000000,
        "isSignal": False,
        "crossSection": 5075.6,
        "color": leg_dict["DY"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/DYJets_DL_2017_PUFix.root",
        },
    "MuMu":{
        "era": "2017",
        "subera": "B",
        "channel": "MuMu",
        "isData": True,
        "nEvents": 14501767 + 49636525 + 23075733 + 51589091 + 79756560,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/MuMu_*_2017.root",
        },
    "ElEl":{
        "era": "2017",
        "subera": "B",
        "channel": "ElEl",
        "isData": True,
        "nEvents": 58088760 + 65181125 + 25911432 + 56233597 + 74307066,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElEl_*_2017.root",
        },
    "ElMu":{
        "era": "2017",
        "subera": "B",
        "channel": "ElMu",
        "isData": True,
        "nEvents": 4453465 + 15595214 + 9164365 + 19043421 + 25776363,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/ElMu_*_2017.root",
        },
    "Mu":{
        "era": "2017",
        "subera": "B",
        "channel": "Mu",
        "isData": True,
        "nEvents": 136300266 + 165652756 + 70361660 + 154630534 + 242135500,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/Mu_*_2017.root",
        },
    "El":{
        "era": "2017",
        "subera": "B",
        "channel": "El",
        "isData": True,
        "nEvents": 60537490 + 136637888 + 51526710 + 102121689 + 128467223,
        "color": leg_dict["Data"],
        "source": "/eos/user/n/nmangane/SWAN_projects/LogicChainRDF/El_*_2017.root",
        },
    }

#Set up channel bits for selection and baseline. Separation not necessary in this stage, but convenient for loops
Chan = {}
Chan["ElMu_selection"] = 24576
Chan["MuMu_selection"] = 6144
Chan["ElEl_selection"] = 512
Chan["Mu_selection"] = 128
Chan["El_selection"] = 64
Chan["selection"] = Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"] + Chan["Mu_selection"] + Chan["El_selection"]
Chan["ElMu_baseline"] = 24576
Chan["MuMu_baseline"] = 6144
Chan["ElEl_baseline"] = 512
Chan["Mu_baseline"] = 128
Chan["El_baseline"] = 64
Chan["baseline"] = Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + Chan["ElEl_baseline"] + Chan["Mu_baseline"] + Chan["El_baseline"]


#samples["tt_DL-GF"] = {}
#samples["tt_DL-GF"]["path"] = base + "crab_tt_DL-GF_2017/results/tree*.root"
#booker["tttt"]['nEvents']

{'ttH': 616L, 'tttt': 858L, 'ttVJets': 880L, 'DY': 432L, 'Data': 1L, 'ttbar': 632L, 'QCD': 900L, 'singletop': 400L, 'ttultrarare': 416L}


In [11]:
print(ROOT.kBlue)

600


In [8]:
transverseMassCode = '''auto MT2 = {m1}*{m1} + {m2}*{m2} + 2*(sqrt({m1}*{m1} + {pt1}*{pt1})*sqrt({m2}*{m2} + {pt2}*{pt2}) - {pt1}*{pt2}*cos({phi1} - {phi2}));
                         return sqrt(MT2);'''
b = transverseMassCode.format(m1 = "GMuon_mass", m2 = "0",
                                                                  pt1 = "GMuon_pt",
                                                                 pt2 = "METFixEE2017_pt", 
                                                                 phi1 = "GMuon_phi", phi2 = "METFixEE2017_phi")
print(b)

auto MT2 = GMuon_mass*GMuon_mass + 0*0 + 2*(sqrt(GMuon_mass*GMuon_mass + GMuon_pt*GMuon_pt)*sqrt(0*0 + METFixEE2017_pt*METFixEE2017_pt) - GMuon_pt*METFixEE2017_pt*cos(GMuon_phi - METFixEE2017_phi));
                         return sqrt(MT2);


In [9]:
def defineLeptons(input_df, input_lvl_filter=None, channel=None, isData=True, useBackupChannel=False):
    """Function to take in a dataframe and return one with new columns defined,
    plus event filtering based on the criteria defined inside the function"""
    if channel == None:
        raise RuntimeError("channel must be selected, such as 'MuMu' or 'ElMu'")
    elif channel == "MuMu":
        nMuExp = 2
        nElExp = 0
    elif channel == "ElMu":
        nMuExp = 1
        nElExp = 1
    elif channel == "MuMu":
        nMuExp = 0
        nElExp = 2
        
    #Set up channel bits for selection and baseline. Separation not necessary in this stage, but convenient for loops
    Chan = {}
    Chan["ElMu_selection"] = 24576
    Chan["MuMu_selection"] = 6144
    Chan["ElEl_selection"] = 512
    Chan["Mu_selection"] = 128
    Chan["El_selection"] = 64
    Chan["selection"] = Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"] + Chan["Mu_selection"] + Chan["El_selection"]
    Chan["ElMu_baseline"] = 24576
    Chan["MuMu_baseline"] = 6144
    Chan["ElEl_baseline"] = 512
    Chan["Mu_baseline"] = 128
    Chan["El_baseline"] = 64
    Chan["baseline"] = Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + Chan["ElEl_baseline"] + Chan["Mu_baseline"] + Chan["El_baseline"]
    b = {}
    b["baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + 
                                                                            Chan["ElEl_baseline"] + Chan["Mu_baseline"] + Chan["El_baseline"])
    
    b["ElMu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) > 0".format(Chan["ElMu_baseline"])
    b["MuMu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"], 
                                                                                                                                Chan["MuMu_baseline"])
    b["ElEl_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"], 
                                                                                                                                Chan["ElEl_baseline"])
    b["Mu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + Chan["ElEl_baseline"], Chan["Mu_baseline"])
    b["El_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + 
                                                                                                                    Chan["ElEl_baseline"] + Chan["Mu_baseline"], Chan["El_baseline"])
    b["selection"] = "ESV_TriggerAndLeptonLogic_selection > 0"
    b["ElMu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) > 0".format(Chan["ElMu_selection"])
    b["MuMu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"], Chan["MuMu_selection"])
    b["ElEl_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"], Chan["ElEl_selection"])
    b["Mu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"], Chan["Mu_selection"])
    b["El_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"]
                                                                                                                                 + Chan["Mu_selection"], Chan["El_selection"])
    if input_lvl_filter == None:
        rdf_input = input_df\
                .Define("mu_mask", "Muon_pt > 0").Define("e_mask", "Electron_pt > 0")
    else:
        if "baseline" in input_lvl_filter:
            lvl_type = "baseline"
        elif "selection" in input_lvl_filter:
            lvl_type = "selection"
        else:
            raise RuntimeError("No such level permissable: must contain 'selection' or 'baseline'")
        rdf_input = input_df.Filter(b[input_lvl_filter], input_lvl_filter)
        rdf = rdf_input
        rdf = rdf.Define("mu_mask", "(Muon_OSV_{0} & {1}) > 0".format(lvl_type, Chan[input_lvl_filter]))
        rdf = rdf.Define("e_mask", "(Electron_OSV_{0} & {1}) > 0".format(lvl_type, Chan[input_lvl_filter]))
    indexDefineCode = '''ROOT::VecOps::RVec<int> i({0}.size());
                     std::iota(i.begin(), i.end(), 0);
                     return i;'''
    transverseMassCode = '''auto MT2 = {m1}*{m1} + {m2}*{m2} + 2*(sqrt({m1}*{m1} + {pt1}*{pt1})*sqrt({m2}*{m2} + {pt2}*{pt2}) - {pt1}*{pt2}*cos(ROOT::VecOps::DeltaPhi({phi1}, {phi2})));
                         return sqrt(MT2);'''
    transverseMassCodeChecker = '''auto V1 = ROOT::Math::PtEtaPhiMVector({pt1}, {eta1}, {phi1}, {m1});
                                auto V2 = ROOT::Math::PtEtaPhiMVector({pt2}, {eta2}, {phi2}, {m2});
                                auto V = V1 + V2;
                                return V.Mt();'''
    transverseMassLightCode = '''auto MT2 = 2*{pt1}*{pt2}*(1 - cos(ROOT::VecOps::DeltaPhi({phi1}, {phi2})));
                              return sqrt(MT2);'''
    rdf = rdf.Define("Muon_idx", indexDefineCode.format("mu_mask"))
    rdf = rdf.Define("GMuon_idx", "Muon_idx[mu_mask]")
    rdf = rdf.Define("GMuon_pfIsoId", "Muon_pfIsoId[mu_mask]")
    rdf = rdf.Define("GMuon_looseId", "Muon_looseId[mu_mask]")
    rdf = rdf.Define("GMuon_pt", "Muon_pt[mu_mask]")
    rdf = rdf.Define("GMuon_eta", "Muon_eta[mu_mask]")
    rdf = rdf.Define("GMuon_phi", "Muon_phi[mu_mask]")
    rdf = rdf.Define("GMuon_mass", "Muon_mass[mu_mask]")
    rdf = rdf.Define("GMuon_charge", "Muon_charge[mu_mask]")
    rdf = rdf.Define("GMuon_dz", "Muon_dz[mu_mask]")
    rdf = rdf.Define("GMuon_dxy", "Muon_dxy[mu_mask]")
    rdf = rdf.Define("GMuon_d0", "sqrt(Muon_dz*Muon_dz + Muon_dxy*Muon_dxy)[mu_mask]")
    rdf = rdf.Define("GMuon_ip3d", "Muon_ip3d[mu_mask]")
    rdf = rdf.Define("GMuon_pfRelIso03_all", "Muon_pfRelIso03_all[mu_mask]")
    rdf = rdf.Define("GMuon_pfRelIso03_chg", "Muon_pfRelIso03_chg[mu_mask]")
    rdf = rdf.Define("GMuon_pfRelIso04_all", "Muon_pfRelIso04_all[mu_mask]")
    rdf = rdf.Define("GMuon_jetIdx", "Muon_jetIdx[mu_mask]")
    rdf = rdf.Define("MTofMETandMu", transverseMassCode.format(m1 = "GMuon_mass", m2 = "0",
                                                                  pt1 = "GMuon_pt",
                                                                 pt2 = "METFixEE2017_pt", 
                                                                 phi1 = "GMuon_phi", phi2 = "METFixEE2017_phi"))
    rdf = rdf.Define("MTMasslessCheck", transverseMassLightCode.format(pt1 = "GMuon_pt.at(0)", pt2 = "METFixEE2017_pt", 
                                                                            phi1 = "GMuon_phi.at(0)", phi2 = "METFixEE2017_phi"))
    rdf = rdf.Define("MTCrossCheck", transverseMassCodeChecker.format(m1 = "GMuon_mass.at(0)", m2 = "0",
                                                                         eta1 = "GMuon_eta.at(0)", eta2 = "0",
                                                                         pt1 = "GMuon_pt.at(0)", pt2 = "METFixEE2017_pt", 
                                                                         phi1 = "GMuon_phi.at(0)", phi2 = "METFixEE2017_phi"))
    rdf = rdf.Define("MTCrossCheckDifference", "abs(MTofMETandMu - MTCrossCheck)/MTCrossCheck")
    rdf = rdf.Define("MTCrossCheckMasslessDifference", "abs(MTMasslessCheck - MTofMETandMu)/MTMasslessCheck")
    rdf = rdf.Define("nGMuon", "GMuon_pt.size()")
    rdf = rdf.Define("nLooseGMuon", "Muon_looseId[mu_mask && Muon_looseId == true].size()")
    rdf = rdf.Define("nMediumGMuon", "Muon_mediumId[mu_mask && Muon_mediumId == true].size()")
    rdf = rdf.Define("nTightGMuon", "Muon_tightId[mu_mask && Muon_tightId == true].size()")
    rdf = rdf.Define("Electron_idx", indexDefineCode.format("e_mask"))
    rdf = rdf.Define("GElectron_idx", "Electron_idx[e_mask]")
    rdf = rdf.Define("GElectron_cutBased", "Electron_cutBased[e_mask]")
    rdf = rdf.Define("GElectron_pt", "Electron_pt[e_mask]")
    rdf = rdf.Define("GElectron_eta", "Electron_eta[e_mask]")
    rdf = rdf.Define("GElectron_phi", "Electron_phi[e_mask]")
    rdf = rdf.Define("GElectron_mass", "Electron_mass[e_mask]")
    rdf = rdf.Define("GElectron_charge", "Electron_charge[e_mask]")
    rdf = rdf.Define("GElectron_dz", "Electron_dz[e_mask]")
    rdf = rdf.Define("GElectron_dxy", "Electron_dxy[e_mask]")
    rdf = rdf.Define("GElectron_d0", "sqrt(Electron_dz*Electron_dz + Electron_dxy*Electron_dxy)[e_mask]")
    rdf = rdf.Define("GElectron_ip3d", "Electron_ip3d[e_mask]")
    rdf = rdf.Define("GElectron_pfRelIso03_all", "Electron_pfRelIso03_all[e_mask]")
    rdf = rdf.Define("GElectron_pfRelIso03_chg", "Electron_pfRelIso03_chg[e_mask]")
    rdf = rdf.Define("GElectron_jetIdx", "Electron_jetIdx[e_mask]")
    rdf = rdf.Define("MTofMETandEl", transverseMassCode.format(m1 = "GElectron_mass", m2 = "0",
                                                                  pt1 = "GElectron_pt",
                                                                 pt2 = "METFixEE2017_pt", 
                                                                 phi1 = "GElectron_phi", phi2 = "METFixEE2017_phi"))
    
    rdf = rdf.Define("MTofElandMu", transverseMassCode.format(m1 = "GElectron_mass", m2 = "GMuon_mass",
                                                                  pt1 = "GElectron_pt",
                                                                 pt2 = "GMuon_pt", 
                                                                 phi1 = "GElectron_phi", phi2 = "GMuon_phi"))
    rdf = rdf.Define("nGElectron", "GElectron_pt.size()")
    rdf = rdf.Define("nLooseGElectron", "Sum(GElectron_cutBased >= 2)")
    rdf = rdf.Define("nMediumGElectron", "Sum(GElectron_cutBased >= 3)")
    rdf = rdf.Define("nTightGElectron", "Sum(GElectron_cutBased >= 4)")
    rdf = rdf.Define("GLepton_argsort", "Reverse(Argsort(Concatenate(Muon_pt[mu_mask], Electron_pt[e_mask])))")
    rdf = rdf.Define("GLepton_pt", "Take(Concatenate(Muon_pt[mu_mask], Electron_pt[e_mask]), GLepton_argsort)")
    rdf = rdf.Define("GLepton_eta", "Take(Concatenate(Muon_eta[mu_mask], Electron_eta[e_mask]), GLepton_argsort)")
    rdf = rdf.Define("GLepton_phi", "Take(Concatenate(Muon_phi[mu_mask], Electron_phi[e_mask]), GLepton_argsort)")
    rdf = rdf.Define("GLepton_jetIdx", "Take(Concatenate(Muon_jetIdx[mu_mask], Electron_jetIdx[e_mask]), GLepton_argsort)")
    rdf = rdf.Define("GLepton_pdgId", "Take(Concatenate(Muon_pdgId[mu_mask], Electron_pdgId[e_mask]), GLepton_argsort)")
    rdf = rdf.Define("GLepton_dRll", "GLepton_pt.size() > 1 ? ROOT::VecOps::DeltaR(GLepton_eta.at(0), GLepton_eta.at(1), GLepton_phi.at(0), GLepton_phi.at(1)) : -0.1")
    rdf = rdf.Define("GLepton_dPhill", "GLepton_pt.size() > 1 ? ROOT::VecOps::DeltaPhi(GLepton_phi.at(0), GLepton_phi.at(1)) : -999")
    rdf = rdf.Define("GLepton_dEtall", "GLepton_pt.size() > 1 ? abs(GLepton_eta.at(0) - GLepton_eta.at(1)) : -999")
    rdf = rdf.Define("nGLepton", "GLepton_pt.size()")
    rdf = rdf.Define("GLepton_pt_LeadLep", "GLepton_pt.size() > 0 ? GLepton_pt.at(0) : -0.000000000001")
    rdf = rdf.Define("GLepton_pt_SubleadLep", "GLepton_pt.size() > 1 ? GLepton_pt.at(1) : -0.000000000001")
    rdf = rdf.Define("GLepton_jetIdx_0", "GLepton_jetIdx.size() > 0 ? GLepton_jetIdx.at(0) : -1")
    rdf = rdf.Define("GLepton_jetIdx_1", "GLepton_jetIdx.size() > 1 ? GLepton_jetIdx.at(1) : -1")
    if isData == False:
        rdf = rdf.Define("GMuon_SF_ID_nom", "Muon_SF_ID_nom[mu_mask]")
        rdf = rdf.Define("GMuon_SF_ID_stat", "Muon_SF_ID_stat[mu_mask]")
        rdf = rdf.Define("GMuon_SF_ID_syst", "Muon_SF_ID_syst[mu_mask]")
        rdf = rdf.Define("GMuon_SF_ISO_nom", "Muon_SF_ISO_nom[mu_mask]")
        rdf = rdf.Define("GMuon_SF_ISO_stat", "Muon_SF_ISO_stat[mu_mask]")
        rdf = rdf.Define("GMuon_SF_ISO_syst", "Muon_SF_ISO_syst[mu_mask]")
        rdf = rdf.Define("GElectron_SF_EFF_nom", "Electron_SF_EFF_nom[e_mask]")
        rdf = rdf.Define("GElectron_SF_EFF_unc", "Electron_SF_EFF_unc[e_mask]")
        rdf = rdf.Define("GElectron_SF_ID_nom", "Electron_SF_ID_nom[e_mask]")
        rdf = rdf.Define("GElectron_SF_ID_unc", "Electron_SF_ID_unc[e_mask]")
        rdf = rdf.Define("GLepton_SF_nom", "Take(Concatenate(Muon_SF_ID_nom[mu_mask]*Muon_SF_ISO_nom[mu_mask], Electron_SF_ID_nom[e_mask]*Electron_SF_EFF_nom[e_mask]), GLepton_argsort)")
#    else:
#        rdf = rdf.Define("GMuon_SF_ID_nom", "1")
#        rdf = rdf.Define("GMuon_SF_ID_stat", "0")
#        rdf = rdf.Define("GMuon_SF_ID_syst", "0")
#        rdf = rdf.Define("GMuon_SF_ISO_nom", "1")
#        rdf = rdf.Define("GMuon_SF_ISO_stat", "0")
#        rdf = rdf.Define("GMuon_SF_ISO_syst", "0")
#        rdf = rdf.Define("GElectron_SF_EFF_nom", "1")
#        rdf = rdf.Define("GElectron_SF_EFF_unc", "0")
#        rdf = rdf.Define("GElectron_SF_ID_nom", "1")
#        rdf = rdf.Define("GElectron_SF_ID_unc", "0")
#        rdf = rdf.Define("GLepton_SF_nom", "Take(Concatenate(Muon_SF_ID_nom[mu_mask]*Muon_SF_ISO_nom[mu_mask], Electron_SF_ID_nom[e_mask]*Electron_SF_EFF_nom[e_mask]), GLepton_argsort)")
    
                
    #Things that don't work...
    #NOPE doesn't work .Define("nLooseGMuon", "Sum(Muon_looseId[mu_mask])")\
    return rdf

In [10]:
def defineWeights(input_df, crossSection=0, sumWeights=-1, lumi=0,
                  nEvents=-1, nEventsPositive=2, nEventsNegative=1,
                  channel=None, isData=True, verbose=False):
    
    mc_def = collections.OrderedDict()
    data_def = collections.OrderedDict()
    mc_def["wgt_NUMW"] = "({xs:s} * {lumi:s} * 1000 * genWeight) / (abs(genWeight) * ( {nevtp:s} - {nevtn:s} ) )"\
            .format(xs=str(crossSection), lumi=str(lumi), nevt=str(nEvents),
                    nevtp=str(nEventsPositive), nevtn=str(nEventsNegative))
    mc_def["wgt_SUMW"] = "({xs:s} * {lumi:s} * 1000 * genWeight) / {sumw:s}"\
            .format(xs=str(crossSection), lumi=str(lumi), sumw=str(sumWeights))
    mc_def["wgt_LSF"] = "(GLepton_SF_nom.size() > 1 ? GLepton_SF_nom.at(0) * GLepton_SF_nom.at(1) : GLepton_SF_nom.at(0))"
    mc_def["wgt_SUMW_PU"] = "wgt_SUMW * puWeight"
    mc_def["wgt_SUMW_LSF"] = "wgt_SUMW * wgt_LSF"
    mc_def["wgt_SUMW_L1PF"] = "wgt_SUMW * L1PreFiringWeight_Nom"
    mc_def["wgt_SUMW_PU_LSF"] = "wgt_SUMW * puWeight * wgt_LSF"
    mc_def["wgt_SUMW_PU_LSF_L1PF"] = "wgt_SUMW * puWeight * wgt_LSF * L1PreFiringWeight_Nom"
    mc_def["wgt_SUMW_LSF_L1PF"] = "wgt_SUMW * wgt_LSF * L1PreFiringWeight_Nom"
    mc_def["wgt_NUMW_LSF_L1PF"] = "wgt_NUMW * wgt_LSF * L1PreFiringWeight_Nom"
    #mc_def["wgt_SUMW_PU_LSF"] = "wgt_SUMW * puWeight * GLepton_SF_nom.at(0) * GLepton_SF_nom.at(1)"
    mc_def["SPL_SP"] = "wgt_SUMW_PU_LSF/wgt_SUMW_PU"
    mc_def["wgt_diff"] = "abs(wgt_NUMW - wgt_SUMW)/max(abs(wgt_SUMW), abs(wgt_NUMW))"
    for k in mc_def.keys():
        data_def[k] = "1"
    if verbose == True:
        print("===data and mc weight definitions===")
        print(data_def)
        print(mc_def)
        
    rdf = input_df
    if isData:
        for k, v in data_def.items():
            rdf = rdf.Define(k, v)
    else:
        for k, v in mc_def.items():
            rdf = rdf.Define(k, v)
    return rdf

In [11]:
def defineJets(input_df, era="2017", doAK8Jets=False, debugInfo = True, nJetsToHisto=10):
    """Function to take in a dataframe and return one with new columns defined,
    plus event filtering based on the criteria defined inside the function"""
    indexDefineCode = '''ROOT::VecOps::RVec<int> i({0}.size());
                     std::iota(i.begin(), i.end(), 0);
                     return i;'''
    bTagWorkingPointDict = {
        '2016':{
            'DeepCSV':{'L': 0.2217, 'M': 0.6321, 'T': 0.8953, 'Var': 'btagDeepB'},
            'DeepJet':{ 'L': 0.0614, 'M': 0.3093, 'T': 0.7221, 'Var': 'btagDeepFlavB'}
        },
        '2017':{
            'CSVv2':{'L': 0.5803, 'M': 0.8838, 'T': 0.9693, 'Var': 'btagCSVV2'},
            'DeepCSV':{'L': 0.1522, 'M': 0.4941, 'T': 0.8001, 'Var': 'btagDeepB'},
            'DeepJet':{'L': 0.0521, 'M': 0.3033, 'T': 0.7489, 'Var': 'btagDeepFlavB'}
        },
        '2018':{
            'DeepCSV':{'L': 0.1241, 'M': 0.4184, 'T': 0.7527, 'Var': 'btagDeepB'},
            'DeepJet':{'L': 0.0494, 'M': 0.2770, 'T': 0.7264, 'Var': 'btagDeepFlavB'}
        }
    }
    print("FIXMEFIXME: Setting Jet_pt min to 30GeV! Must fix!")
    rdf = input_df
    rdf = rdf.Define("jet_maskALT", "(Jet_OSV_baseline & {0}) > {0}".format(23))
    rdf = rdf.Define("jet_submask", "(Jet_pt >= 30 && abs(Jet_eta) <= 2.5 && Jet_jetId > 2)")
    rdf = rdf.Define("Jet_idx", indexDefineCode.format("Jet_pt"))
    rdf = rdf.Define("jet_mask", "(jet_submask && Jet_idx != GLepton_jetIdx_0 && Jet_idx != GLepton_jetIdx_1)")
    rdf = rdf.Define("jet_ptsort", "Reverse(Argsort(Jet_pt[jet_mask]))")
    rdf = rdf.Define("jet_deepcsvsort", "Reverse(Argsort(Jet_{0}[jet_mask]))".format(bTagWorkingPointDict[era]["DeepCSV"]["Var"]))
    rdf = rdf.Define("jet_deepjetsort", "Reverse(Argsort(Jet_{0}[jet_mask]))".format(bTagWorkingPointDict[era]["DeepJet"]["Var"]))
    rdf = rdf.Define("GJet_idx", "Jet_idx[jet_mask]")
    rdf = rdf.Define("GJet_pt", "Jet_pt[jet_mask]")
    for x in xrange(nJetsToHisto):
        rdf = rdf.Define("GJet_pt_jet{}".format(x+1), "GJet_pt.size() > {} ? GJet_pt.at({}) : -9999".format(x, x))
    rdf = rdf.Define("GJet_eta", "Jet_eta[jet_mask]")
    rdf = rdf.Define("GJet_phi", "Jet_phi[jet_mask]")
    rdf = rdf.Define("GJet_mass", "Jet_mass[jet_mask]")
    rdf = rdf.Define("GJet_jetId", "Jet_jetId[jet_mask]")
    rdf = rdf.Define("nGJet", "GJet_pt.size()")
    rdf = rdf.Define("GJet_btagDeepB", "Jet_btagDeepB[jet_mask]")
    rdf = rdf.Define("GJet_btagDeepB_LeadtagJet", "GJet_btagDeepB.size() > 0 ? Reverse(Sort(GJet_btagDeepB)).at(0) : -0.000000000001")
    rdf = rdf.Define("GJet_btagDeepB_SubleadtagJet", "GJet_btagDeepB.size() > 1 ? Reverse(Sort(GJet_btagDeepB)).at(1) : -0.000000000001")
    rdf = rdf.Define("GJet_btagDeepFlavB", "Jet_btagDeepFlavB[jet_mask]")
    rdf = rdf.Define("GJet_btagDeepFlavB_sorted", "Take(Jet_btagDeepFlavB[jet_mask], jet_deepjetsort)")
    rdf = rdf.Define("GJet_btagDeepFlavB_sorted_LeadtagJet", "GJet_btagDeepFlavB_sorted.size() > 0 ? GJet_btagDeepFlavB_sorted.at(0) : -0.000000000001")
    rdf = rdf.Define("GJet_btagDeepFlavB_sorted_SubleadtagJet", "GJet_btagDeepFlavB_sorted.size() > 1 ? GJet_btagDeepFlavB_sorted.at(1) : -0.000000000001")
    rdf = rdf.Define("GJet_MediumDeepCSV", "GJet_{0}[GJet_{0} > {1}]".format(bTagWorkingPointDict[era]["DeepCSV"]["Var"],
                                                                                bTagWorkingPointDict[era]["DeepCSV"]["M"]))
    rdf = rdf.Define("nGJet_MediumDeepCSV", "GJet_MediumDeepCSV.size()")
    rdf = rdf.Define("GJet_MediumDeepJet", "GJet_{0}[GJet_{0} > {1}]".format(bTagWorkingPointDict[era]["DeepJet"]["Var"],
                                                                                bTagWorkingPointDict[era]["DeepJet"]["M"]))
    rdf = rdf.Define("nGJet_MediumDeepJet", "GJet_MediumDeepJet.size()")
    #These might be more efficiently calculated with my own custom code, instead of this... well, lets try for the sake of experimentation
    #HT is just the sum of good jet pts
    # HT2M is the sum of jet pt's for all but the two highest-b-tagged jets (2016 analysis requires 4+ jets to define this quantity), so here Take() is used twice.
    # The first call acquires the good jet pt's sorted by b-tagging, the senond Take() gets the last n-2 elements, thereby excluding the two highest b-tagged jet's pt
    # HTRat = HT(two highest b-tagged) / HT, so it's useful to define this similarly to HT2M (and crosscheck that HTNum + HT2M = HT!)
    # H and H2M are defined similarly for the overall momentum magnitude...
    # P = pt/sin(theta) = pt * (1/sin(theta)) = pt * cosh(eta)
    rdf = rdf.Define("GJet_HT", "Sum(GJet_pt)")
    rdf = rdf.Define("GJet_pt_bsrt", "Take(GJet_pt, jet_deepjetsort)")
    rdf = rdf.Define("GJet_eta_bsrt", "Take(GJet_eta, jet_deepjetsort)")
    rdf = rdf.Define("GJet_phi_bsrt", "Take(GJet_phi, jet_deepjetsort)")
    rdf = rdf.Define("GJet_P_bsrt", "GJet_pt_bsrt * ROOT::VecOps::cosh(GJet_eta_bsrt)")
    rdf = rdf.Define("GJet_HT2M", "GJet_pt_bsrt.size() > 2 ? Sum(Take(GJet_pt_bsrt, (2 - GJet_pt_bsrt.size()))) : -0.1")
    rdf = rdf.Define("GJet_HTNum", "GJet_pt_bsrt.size() > 2 ? Sum(Take(GJet_pt_bsrt, 2)) : -0.1")
    rdf = rdf.Define("GJet_HTRat", "GJet_pt_bsrt.size() > 2 ? (GJet_HT2M / GJet_HT) : -0.1")
    rdf = rdf.Define("GJet_dRbb", "GJet_pt_bsrt.size() > 2 ? ROOT::VecOps::DeltaR(GJet_eta_bsrt.at(0), GJet_eta_bsrt.at(1), GJet_phi_bsrt.at(0), GJet_phi_bsrt.at(1)) : -0.1")
    rdf = rdf.Define("GJet_dPhibb", "GJet_pt_bsrt.size() > 2 ? ROOT::VecOps::DeltaPhi(GJet_phi_bsrt.at(0), GJet_phi_bsrt.at(1)) : -999")
    rdf = rdf.Define("GJet_dEtabb", "GJet_pt_bsrt.size() > 2 ? abs(GJet_eta_bsrt.at(0) - GJet_eta_bsrt.at(1)) : -999")
    rdf = rdf.Define("GJet_H", "Sum(GJet_P_bsrt)")
    rdf = rdf.Define("GJet_H2M", "GJet_pt_bsrt.size() > 2 ? Sum(Take(GJet_P_bsrt, (2 - GJet_pt_bsrt.size()))) : -0.1")
    rdf = rdf.Define("GJet_HTH", "GJet_HT/GJet_H")
    rdf = rdf.Define("GJet_HTb", "Sum(GJet_pt[GJet_{0} > {1}])".format(bTagWorkingPointDict[era]["DeepJet"]["Var"],
                                                                                bTagWorkingPointDict[era]["DeepJet"]["M"]))
    if debugInfo == True:
        rdf = rdf.Define("GJet_ptALT", "Jet_pt[jet_maskALT]")
        rdf = rdf.Define("GJet_etaALT", "Jet_eta[jet_maskALT]")
        rdf = rdf.Define("GJet_phiALT", "Jet_phi[jet_maskALT]")
        rdf = rdf.Define("GJet_massALT", "Jet_mass[jet_maskALT]")
        rdf = rdf.Define("GJet_jetIdALT", "Jet_jetId[jet_maskALT]")
        rdf = rdf.Define("DiffMaskVsALT", "GJet_ptALT.size() - GJet_pt.size()")
        rdf = rdf.Define("DiffnJet", "nGJet - ESV_JetMETLogic_nJet_selection")
        rdf = rdf.Define("dR_Jet_Mu_leading", "GLepton_jetIdx_0 > -1 && abs(GLepton_pdgId.at(0)) == 13 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_0), GLepton_eta.at(0), Jet_phi.at(GLepton_jetIdx_0), GLepton_phi.at(0)) : -0.01")
        rdf = rdf.Define("dR_Jet_Mu_sublead", "GLepton_jetIdx_1 > -1 && abs(GLepton_pdgId.at(1)) == 13 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_1), GLepton_eta.at(1), Jet_phi.at(GLepton_jetIdx_1), GLepton_phi.at(1)) : -0.01")
        rdf = rdf.Define("dR_Jet_El_leading", "GLepton_jetIdx_0 > -1 && abs(GLepton_pdgId.at(1)) == 11 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_0), GLepton_eta.at(0), Jet_phi.at(GLepton_jetIdx_0), GLepton_phi.at(0)) : -0.01")
        rdf = rdf.Define("dR_Jet_El_sublead", "GLepton_jetIdx_1 > -1 && abs(GLepton_pdgId.at(1)) == 11 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_1), GLepton_eta.at(1), Jet_phi.at(GLepton_jetIdx_1), GLepton_phi.at(1)) : -0.01")
        #rdf = rdf.Define("dR_Jet_lep0", "GLepton_jetIdx_0 > -1 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_0), GLepton_eta.at(0), Jet_phi.at(GLepton_jetIdx_0), GLepton_phi.at(0)) : -0.01")
        #rdf = rdf.Define("dR_Jet_lep1", "GLepton_jetIdx_1 > -1 ? ROOT::VecOps::DeltaR(Jet_eta.at(GLepton_jetIdx_1), GLepton_eta.at(1), Jet_phi.at(GLepton_jetIdx_1), GLepton_phi.at(1)) : -0.01")
        rdf = rdf.Define("GJet_btagDeepFlavB_jet0Med", "GJet_MediumDeepJet.size() > 0 ? Reverse(Sort(GJet_MediumDeepJet)).at(0) : -0.000000000001")
        rdf = rdf.Define("GJet_btagDeepFlavB_jet1Med", "GJet_MediumDeepJet.size() > 1 ? Reverse(Sort(GJet_MediumDeepJet)).at(1) : -0.000000000001")
        rdf = rdf.Define("DeepJetSorted", "GJet_btagDeepFlavB_sorted.size() > 1 ? (GJet_btagDeepFlavB_sorted.at(0) >= GJet_btagDeepFlavB_sorted.at(1)) : true")
        rdf = rdf.Define("DeepJet0Minus1", "GJet_btagDeepFlavB_sorted.size() > 1 ? (GJet_btagDeepFlavB_sorted.at(0) - GJet_btagDeepFlavB_sorted.at(1)) : 0")
        rdf = rdf.Define("MediumDeepJetSorted", "GJet_MediumDeepJet.size() > 1 ? (Reverse(Sort(GJet_MediumDeepJet)).at(0) >= Reverse(Sort(GJet_MediumDeepJet)).at(1)) : true")
        rdf = rdf.Define("MediumDeepJet0Minus1", "GJet_MediumDeepJet.size() > 1 ? (Reverse(Sort(GJet_MediumDeepJet)).at(0) - Reverse(Sort(GJet_MediumDeepJet)).at(1)) : 0")
    return rdf
    #Code taht doesn't work...
    #Can see that the jets are in fact not sorted when calling Reverse(GJet_MediumDeepJet).at(0), for example, as the one .at(1) can sometimes not be smaller
    #Looking at the definition makes it obvious, because Reverse is not short for "ReverseSort" but is literally just std::reverse. Must call (Arg)sort first...
    #Definig a functor in the string like this doesn't work either:
    #.Define("GJet_btagDeepB_jet0", "GJet_btagDeepB.size() > 0 ? Sort(GJet_btagDeepB, [](double x, double y) {return x > y;}).at(0) : -0.000000000001")\
    #Cannot use ternary operator with RVec and double return types (Take(...) : -0.0000000001)
    #.Define("GJet_btagDeepFlavB_jet0", "GJet_btagDeepFlavB.size() > 0 ? Take(Reverse(GJet_btagDeepFlavB), {0}) : -0.000000000001")\

In [12]:
def MET_corrections(input_df, uncormet_pt_branch = "METFixEE2017_pt", uncormet_phi_branch = "METFixEE2017_phi", 
                    run_branch = "run", year = "2017", isMC = False, npv_branch = ""):
    MET_CorrXY_Code = '''
enum TheRunEra{y2016B,y2016C,y2016D,y2016E,y2016F,y2016G,y2016H,y2017B,y2017C,y2017D,y2017E,y2017F,y2018A,y2018B,y2018C,y2018D,y2016MC,y2017MC,y2018MC};

std::pair<double,double> METXYCorr_Met_MetPhi(double uncormet, double uncormet_phi, int runnb, int year, bool isMC, int npv){

  std::pair<double,double>  TheXYCorr_Met_MetPhi(uncormet,uncormet_phi);
  
  if(npv>100) npv=100;
  int runera =-1;
  bool usemetv2 =false;
  if(isMC && year == 2016) runera = y2016MC;
  else if(isMC && year == 2017) {runera = y2017MC; usemetv2 =true;}
  else if(isMC && year == 2018) runera = y2018MC;
  
  else if(!isMC && runnb >=272007 &&runnb<=275376  ) runera = y2016B;
  else if(!isMC && runnb >=275657 &&runnb<=276283  ) runera = y2016C;
  else if(!isMC && runnb >=276315 &&runnb<=276811  ) runera = y2016D;
  else if(!isMC && runnb >=276831 &&runnb<=277420  ) runera = y2016E;
  else if(!isMC && runnb >=277772 &&runnb<=278808  ) runera = y2016F;
  else if(!isMC && runnb >=278820 &&runnb<=280385  ) runera = y2016G;
  else if(!isMC && runnb >=280919 &&runnb<=284044  ) runera = y2016H;
  
  else if(!isMC && runnb >=297020 &&runnb<=299329 ){ runera = y2017B; usemetv2 =true;}
  else if(!isMC && runnb >=299337 &&runnb<=302029 ){ runera = y2017C; usemetv2 =true;}
  else if(!isMC && runnb >=302030 &&runnb<=303434 ){ runera = y2017D; usemetv2 =true;}
  else if(!isMC && runnb >=303435 &&runnb<=304826 ){ runera = y2017E; usemetv2 =true;}
  else if(!isMC && runnb >=304911 &&runnb<=306462 ){ runera = y2017F; usemetv2 =true;}
  
  else if(!isMC && runnb >=315252 &&runnb<=316995 ) runera = y2018A;
  else if(!isMC && runnb >=316998 &&runnb<=319312 ) runera = y2018B;
  else if(!isMC && runnb >=319313 &&runnb<=320393 ) runera = y2018C;
  else if(!isMC && runnb >=320394 &&runnb<=325273 ) runera = y2018D;

  else {
    //Couldn't find data/MC era => no correction applied
    return TheXYCorr_Met_MetPhi;
  }
  
  double METxcorr(0.),METycorr(0.);

  if(!usemetv2){//Current recommendation for 2016 and 2018
    if(runera==y2016B) METxcorr = -(-0.0478335*npv -0.108032);
    if(runera==y2016B) METycorr = -(0.125148*npv +0.355672);
    if(runera==y2016C) METxcorr = -(-0.0916985*npv +0.393247);
    if(runera==y2016C) METycorr = -(0.151445*npv +0.114491);
    if(runera==y2016D) METxcorr = -(-0.0581169*npv +0.567316);
    if(runera==y2016D) METycorr = -(0.147549*npv +0.403088);
    if(runera==y2016E) METxcorr = -(-0.065622*npv +0.536856);
    if(runera==y2016E) METycorr = -(0.188532*npv +0.495346);
    if(runera==y2016F) METxcorr = -(-0.0313322*npv +0.39866);
    if(runera==y2016F) METycorr = -(0.16081*npv +0.960177);
    if(runera==y2016G) METxcorr = -(0.040803*npv -0.290384);
    if(runera==y2016G) METycorr = -(0.0961935*npv +0.666096);
    if(runera==y2016H) METxcorr = -(0.0330868*npv -0.209534);
    if(runera==y2016H) METycorr = -(0.141513*npv +0.816732);
    if(runera==y2017B) METxcorr = -(-0.259456*npv +1.95372);
    if(runera==y2017B) METycorr = -(0.353928*npv -2.46685);
    if(runera==y2017C) METxcorr = -(-0.232763*npv +1.08318);
    if(runera==y2017C) METycorr = -(0.257719*npv -1.1745);
    if(runera==y2017D) METxcorr = -(-0.238067*npv +1.80541);
    if(runera==y2017D) METycorr = -(0.235989*npv -1.44354);
    if(runera==y2017E) METxcorr = -(-0.212352*npv +1.851);
    if(runera==y2017E) METycorr = -(0.157759*npv -0.478139);
    if(runera==y2017F) METxcorr = -(-0.232733*npv +2.24134);
    if(runera==y2017F) METycorr = -(0.213341*npv +0.684588);
    if(runera==y2018A) METxcorr = -(0.362865*npv -1.94505);
    if(runera==y2018A) METycorr = -(0.0709085*npv -0.307365);
    if(runera==y2018B) METxcorr = -(0.492083*npv -2.93552);
    if(runera==y2018B) METycorr = -(0.17874*npv -0.786844);
    if(runera==y2018C) METxcorr = -(0.521349*npv -1.44544);
    if(runera==y2018C) METycorr = -(0.118956*npv -1.96434);
    if(runera==y2018D) METxcorr = -(0.531151*npv -1.37568);
    if(runera==y2018D) METycorr = -(0.0884639*npv -1.57089);
    if(runera==y2016MC) METxcorr = -(-0.195191*npv -0.170948);
    if(runera==y2016MC) METycorr = -(-0.0311891*npv +0.787627);
    if(runera==y2017MC) METxcorr = -(-0.217714*npv +0.493361);
    if(runera==y2017MC) METycorr = -(0.177058*npv -0.336648);
    if(runera==y2018MC) METxcorr = -(0.296713*npv -0.141506);
    if(runera==y2018MC) METycorr = -(0.115685*npv +0.0128193);
  }
  else {//these are the corrections for v2 MET recipe (currently recommended for 2017)
    if(runera==y2016B) METxcorr = -(-0.0374977*npv +0.00488262);
    if(runera==y2016B) METycorr = -(0.107373*npv +-0.00732239);
    if(runera==y2016C) METxcorr = -(-0.0832562*npv +0.550742);
    if(runera==y2016C) METycorr = -(0.142469*npv +-0.153718);
    if(runera==y2016D) METxcorr = -(-0.0400931*npv +0.753734);
    if(runera==y2016D) METycorr = -(0.127154*npv +0.0175228);
    if(runera==y2016E) METxcorr = -(-0.0409231*npv +0.755128);
    if(runera==y2016E) METycorr = -(0.168407*npv +0.126755);
    if(runera==y2016F) METxcorr = -(-0.0161259*npv +0.516919);
    if(runera==y2016F) METycorr = -(0.141176*npv +0.544062);
    if(runera==y2016G) METxcorr = -(0.0583851*npv +-0.0987447);
    if(runera==y2016G) METycorr = -(0.0641427*npv +0.319112);
    if(runera==y2016H) METxcorr = -(0.0706267*npv +-0.13118);
    if(runera==y2016H) METycorr = -(0.127481*npv +0.370786);
    if(runera==y2017B) METxcorr = -(-0.19563*npv +1.51859);
    if(runera==y2017B) METycorr = -(0.306987*npv +-1.84713);
    if(runera==y2017C) METxcorr = -(-0.161661*npv +0.589933);
    if(runera==y2017C) METycorr = -(0.233569*npv +-0.995546);
    if(runera==y2017D) METxcorr = -(-0.180911*npv +1.23553);
    if(runera==y2017D) METycorr = -(0.240155*npv +-1.27449);
    if(runera==y2017E) METxcorr = -(-0.149494*npv +0.901305);
    if(runera==y2017E) METycorr = -(0.178212*npv +-0.535537);
    if(runera==y2017F) METxcorr = -(-0.165154*npv +1.02018);
    if(runera==y2017F) METycorr = -(0.253794*npv +0.75776);
    if(runera==y2018A) METxcorr = -(0.362642*npv +-1.55094);
    if(runera==y2018A) METycorr = -(0.0737842*npv +-0.677209);
    if(runera==y2018B) METxcorr = -(0.485614*npv +-2.45706);
    if(runera==y2018B) METycorr = -(0.181619*npv +-1.00636);
    if(runera==y2018C) METxcorr = -(0.503638*npv +-1.01281);
    if(runera==y2018C) METycorr = -(0.147811*npv +-1.48941);
    if(runera==y2018D) METxcorr = -(0.520265*npv +-1.20322);
    if(runera==y2018D) METycorr = -(0.143919*npv +-0.979328);
    if(runera==y2016MC) METxcorr = -(-0.159469*npv +-0.407022);
    if(runera==y2016MC) METycorr = -(-0.0405812*npv +0.570415);
    if(runera==y2017MC) METxcorr = -(-0.182569*npv +0.276542);
    if(runera==y2017MC) METycorr = -(0.155652*npv +-0.417633);
    if(runera==y2018MC) METxcorr = -(0.299448*npv +-0.13866);
    if(runera==y2018MC) METycorr = -(0.118785*npv +0.0889588);
  }

  double CorrectedMET_x = uncormet *cos( uncormet_phi)+METxcorr;
  double CorrectedMET_y = uncormet *sin( uncormet_phi)+METycorr;

  double CorrectedMET = sqrt(CorrectedMET_x*CorrectedMET_x+CorrectedMET_y*CorrectedMET_y);
  double CorrectedMETPhi;
  if(CorrectedMET_x==0 && CorrectedMET_y>0) CorrectedMETPhi = TMath::Pi();
  else if(CorrectedMET_x==0 && CorrectedMET_y<0 )CorrectedMETPhi = -TMath::Pi();
  else if(CorrectedMET_x >0) CorrectedMETPhi = TMath::ATan(CorrectedMET_y/CorrectedMET_x);
  else if(CorrectedMET_x <0&& CorrectedMET_y>0) CorrectedMETPhi = TMath::ATan(CorrectedMET_y/CorrectedMET_x) + TMath::Pi();
  else if(CorrectedMET_x <0&& CorrectedMET_y<0) CorrectedMETPhi = TMath::ATan(CorrectedMET_y/CorrectedMET_x) - TMath::Pi();
  else CorrectedMETPhi =0;

  TheXYCorr_Met_MetPhi.first= CorrectedMET;
  TheXYCorr_Met_MetPhi.second= CorrectedMETPhi;
  return TheXYCorr_Met_MetPhi;

}
'''
    #rdf = input_df
    #rdf = rdf.Define()

In [13]:
def defineEventVars(input_df):
    rdf = input_df
    rdf = rdf.Define("JML_baseline_pass", "(ESV_JetMETLogic_baseline & {0}) >= {0}".format(0b00001100011111111111))
    rdf = rdf.Define("JML_selection_pass", "(ESV_JetMETLogic_selection & {0}) >= {0}".format(0b00001100011111111111))
    return rdf

In [14]:
def fillHistos(input_df, input_name=None, wgtVar="wgt_SUMW", isData = True, histos1D_dict=None, histos2D_dict=None, histosNS_dict=None, 
               doMuons=False, doElectrons=False, doLeptons=False, doJets=False, doWeights=False, doEventVars=False, 
               makeMountains=False, debugInfo=True, nJetsToHisto=10):
    """Method to fill histograms given an input RDataFrame, input sample/dataset name, input histogram dictionaries.
    Has several options of which histograms to fill, such as Leptons, Jets, Weights, EventVars, etc.
    Types of histograms (1D, 2D, those which will not be stacked(NS - histosNS)) are filled by passing non-None
    value to that histosXX_dict variable. Internally stored with structure separating the categories of histos,
    with 'Muons,' 'Electrons,' 'Leptons,' 'Jets,' 'EventVars,' 'Weights' subcategories."""
    if doMuons == False and doElectrons == False and doLeptons == False\
                and doJets==False and doWeights==False and doEventVars==False:
        raise RuntimeError("Must select something to plot:"\
                               "Set do{Muons,Electrons,Leptons,Jets,Weights,EventVars,etc} = True in init method")
    
    pi = ROOT.TMath.Pi()
    if doWeights == True:
        if histosNS_dict != None:
            if "EventVars" not in histosNS_dict:
                histosNS_dict["EventVars"] = {}
            histosNS[name][lvl]["EventVars"]["wgt_NUMW"] = input_df.Histo1D("wgt_NUMW")
            histosNS[name][lvl]["EventVars"][wgtVar] = input_df.Histo1D(wgtVar)
        if histos1D_dict != None:
            if "EventVars" not in histos1D_dict:
                histos1D_dict["EventVars"] = {}
            histos1D_dict["EventVars"]["wgt_diff"] = input_df.Histo1D(("wgt_diff", "(wgt_NUMW - wgt_SUMW)/wgt_SUMW", 2000, -1, 1), "wgt_diff", "1")
            histos1D_dict["EventVars"]["wgt_PU"] = input_df.Histo1D(("wgt_PU", "", 2000, 0, 5), "puWeight", "wgt_SUMW")
            histos1D_dict["EventVars"]["wgt_LSF"] = input_df.Histo1D(("wgt_LSF", "", 2000, 0, 5), "wgt_LSF", "wgt_SUMW")
            histos1D_dict["EventVars"]["wgt_L1PF"] = input_df.Histo1D(("wgt_L1PF", "", 2000, 0, 5), "L1PreFiringWeight_Nom", "wgt_SUMW")
            histos1D_dict["EventVars"]["wgt_PU_LSF_L1PF"] = input_df.Histo1D(("wgt_PU_LSF_L1PF", "", 2000, 0, 5), "wgt_PU_LSF_L1PF", "wgt_SUMW")
    if doMuons == True:
        if histos1D_dict != None:
            if "Muons" not in histos1D_dict: 
                histos1D_dict["Muons"] = {}
            histos1D_dict["Muons"]["idx"] = input_df.Histo1D(("idx_({})".format(wgtVar), "", 5, 0, 5), "Muon_idx", wgtVar)
            histos1D_dict["Muons"]["Gidx"] = input_df.Histo1D(("Gidx_({})".format(wgtVar), "", 5, 0, 5), "GMuon_idx", wgtVar)
            histos1D_dict["Muons"]["nMu"] = input_df.Histo1D(("nMuon_({})".format(wgtVar), "", 5, 0, 5), "nGMuon", wgtVar)
            histos1D_dict["Muons"]["nLooseMu"] = input_df.Histo1D(("nLooseMuon_({})".format(wgtVar), "", 5, 0, 5), "nLooseGMuon", wgtVar)
            histos1D_dict["Muons"]["nMediumMu"] = input_df.Histo1D(("nMediumMuon_({})".format(wgtVar), "", 5, 0, 5), "nMediumGMuon", wgtVar)
            histos1D_dict["Muons"]["pt"] = input_df.Histo1D(("Muon_pt_({})".format(wgtVar), "", 100, 0, 500), "GMuon_pt", wgtVar)
            histos1D_dict["Muons"]["eta"] = input_df.Histo1D(("Muon_eta_({})".format(wgtVar), "", 104, -2.6, 2.6), "GMuon_eta", wgtVar)
            histos1D_dict["Muons"]["phi"] = input_df.Histo1D(("Muon_phi_({})".format(wgtVar), "", 64, -pi, pi), "GMuon_phi", wgtVar)
            #histos1D_dict["Muons"]["mass"] = input_df.Histo1D(("Muon_mass_({})".format(wgtVar), "", 50, 0, 1), "GMuon_mass", wgtVar)
            histos1D_dict["Muons"]["iso"] = input_df.Histo1D(("Muon_iso_({})".format(wgtVar), "", 8, 0, 8), "GMuon_pfIsoId", wgtVar)
            histos1D_dict["Muons"]["dz"] = input_df.Histo1D(("Muon_dz_({})".format(wgtVar), "", 100, -0.01, 0.01), "GMuon_dz", wgtVar)
            histos1D_dict["Muons"]["dxy"] = input_df.Histo1D(("Muon_dxy_({})".format(wgtVar), "", 100, -0.1, 0.1), "GMuon_dxy", wgtVar)
            #histos1D_dict["Muons"]["d0"] = input_df.Histo1D(("Muon_d0_({})".format(wgtVar), "", 100, -0.01, 0.01), "GMuon_d0", wgtVar)
            histos1D_dict["Muons"]["ip3d"] = input_df.Histo1D(("Muon_ip3d_({})".format(wgtVar), "", 100, 0, 0.01), "GMuon_ip3d", wgtVar)
            histos1D_dict["Muons"]["pfRelIso03_all"] = input_df.Histo1D(("Muon_pfRelIso03_all_({})".format(wgtVar), "", 100, 0, 0.2), "GMuon_pfRelIso03_all", wgtVar)
            histos1D_dict["Muons"]["pfRelIso03_chg"] = input_df.Histo1D(("Muon_pfRelIso03_chg_({})".format(wgtVar), "", 100, 0, 0.2), "GMuon_pfRelIso03_chg", wgtVar)
            histos1D_dict["Muons"]["pfRelIso04_all"] = input_df.Histo1D(("Muon_pfRelIso04_all_({})".format(wgtVar), "", 100, 0, 0.2), "GMuon_pfRelIso04_all", wgtVar)
        if histos2D_dict != None:
            if "Muons" not in histos2D_dict:
                histos2D_dict["Muons"] = {}
            histos2D_dict["Muons"]["eta_phi"] = input_df.Histo2D(("Muon_eta_phi_({})".format(wgtVar), "",
                                                                  104, -2.6, 2.6,
                                                                  64, -pi, pi),
                                                                 "GMuon_eta", "GMuon_phi", wgtVar)
            histos2D_dict["Muons"]["dz_ip3d"] = input_df.Histo2D(("Muon_dz_ip3d_({})".format(wgtVar), "",
                                                                  100, -0.01, 0.01,
                                                                  100, 0, 0.01),
                                                                 "GMuon_dz", "GMuon_ip3d", wgtVar)
    if doElectrons == True:
        if histos1D_dict != None:
            if "Electrons" not in histos1D_dict: 
                histos1D_dict["Electrons"] = {}
            histos1D_dict["Electrons"]["nEl"] = input_df.Histo1D(("nElectron_({})".format(wgtVar), "", 5, 0, 5), "nGElectron", wgtVar)
            histos1D_dict["Electrons"]["nLooseEl"] = input_df.Histo1D(("nLooseElectron_({})".format(wgtVar), "", 5, 0, 5), "nLooseGElectron", wgtVar)
            histos1D_dict["Electrons"]["nMediumEl"] = input_df.Histo1D(("nMediumElectron_({})".format(wgtVar), "", 5, 0, 5), "nMediumGElectron", wgtVar)
            histos1D_dict["Electrons"]["pt"] = input_df.Histo1D(("Electron_pt_({})".format(wgtVar), "", 100, 0, 500), "GElectron_pt", wgtVar)
            histos1D_dict["Electrons"]["eta"] = input_df.Histo1D(("Electron_eta_({})".format(wgtVar), "", 104, -2.6, 2.6), "GElectron_eta", wgtVar)
            histos1D_dict["Electrons"]["phi"] = input_df.Histo1D(("Electron_phi_({})".format(wgtVar), "", 64, -pi, pi), "GElectron_phi", wgtVar)
            #histos1D_dict["Electrons"]["mass"] = input_df.Histo1D(("Electron_mass_({})".format(wgtVar), "", 50, 0, 1), "GElectron_mass", wgtVar)
            histos1D_dict["Electrons"]["dz"] = input_df.Histo1D(("Electron_dz_({})".format(wgtVar), "", 100, -0.01, 0.01), "GElectron_dz", wgtVar)
            #histos1D_dict["Electrons"]["d0"] = input_df.Histo1D(("Electron_d0_({})".format(wgtVar), "", 100, 0, 0.01), "GElectron_d0", wgtVar)
            histos1D_dict["Electrons"]["ip3d"] = input_df.Histo1D(("Electron_ip3d_({})".format(wgtVar), "", 100, 0, 0.01), "GElectron_ip3d", wgtVar)
            histos1D_dict["Electrons"]["pfRelIso03_all"] = input_df.Histo1D(("Electron_pfRelIso03_all_({})".format(wgtVar), "", 100, 0, 0.2), "GElectron_pfRelIso03_all", wgtVar)
            histos1D_dict["Electrons"]["pfRelIso03_chg"] = input_df.Histo1D(("Electron_pfRelIso03_chg_({})".format(wgtVar), "", 100, 0, 0.2), "GElectron_pfRelIso03_chg", wgtVar)
            histos1D_dict["Electrons"]["cutBased"] = input_df.Histo1D(("Electron_cutBased_({})".format(wgtVar), "", 5, 0, 5), "GElectron_cutBased", wgtVar)
        if histos2D_dict != None:
            if "Electrons" not in histos2D_dict: 
                histos2D_dict["Electrons"] = {}
            histos2D_dict["Electrons"]["eta_phi"] = input_df.Histo2D(("Electron_eta_phi_({})".format(wgtVar), "",
                                                                      104, -2.6, 2.6,
                                                                      64, -pi, pi),
                                                                     "GElectron_eta", "GElectron_phi", wgtVar)
            histos2D_dict["Electrons"]["dz_ip3d"] = input_df.Histo2D(("Electron_dz_ip3d_({})".format(wgtVar), "",
                                                                      100, -0.01, 0.01,
                                                                      100, 0, 0.01),
                                                                     "GElectron_dz", "GElectron_ip3d", wgtVar)
    if doLeptons == True:
        if histos1D_dict != None:
            if "Leptons" not in histos1D_dict: 
                histos1D_dict["Leptons"] = {}
            histos1D_dict["Leptons"]["pt_LeadLep"] = input_df\
                    .Histo1D(("GLepton_pt_LeadLep_({})".format(wgtVar), "", 100, 0, 500),"GLepton_pt_LeadLep", wgtVar)
            histos1D_dict["Leptons"]["pt_SubleadLep"] = input_df\
                    .Histo1D(("GLepton_pt_SubleadLep_({})".format(wgtVar), "", 100, 0, 500),"GLepton_pt_SubleadLep", wgtVar)
            histos1D_dict["Leptons"]["eta"] = input_df\
                    .Histo1D(("GLepton_eta_({})".format(wgtVar), "", 104, -2.6, 2.6),"GLepton_eta", wgtVar)
            histos1D_dict["Leptons"]["phi"] = input_df\
                    .Histo1D(("GLepton_phi_({})".format(wgtVar), "", 64, -pi, pi),"GLepton_phi", wgtVar)
            histos1D_dict["Leptons"]["nLepton"] = input_df\
                    .Histo1D(("nLepton_({})".format(wgtVar), "", 5, 0, 5), "nGLepton", wgtVar)
            histos1D_dict["Leptons"]["pdgId"] = input_df\
                    .Histo1D(("Lepton_pdgId_({})".format(wgtVar), "", 32, -16, 16), "GLepton_pdgId", wgtVar)
            histos1D_dict["Leptons"]["jetIdx"] = input_df\
                    .Histo1D(("Lepton_jetIdx_({})".format(wgtVar), "", 20, 0, 20), "GLepton_jetIdx", wgtVar)
            #histos1D_dict["Leptons"]["LepSF"] = input_df\
            #        .Histo1D(("Lepton_SF_({})".format("wgt_SUMW_PU:HARDCODED"), "", 100, 0.93, 1.03), "GLepton_SF_nom", "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["LSF"] = input_df\
            #        .Histo1D(("LSF_({})".format("wgt_SUMW_PU:HARDCODED"), "", 200, 0.80, 1.1), "wgt_LSF", "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["SPL_SP"] = input_df\
            #        .Histo1D(("SPL_SP_({})".format("wgt_SUMW_PU:HARDCODED"), "", 200, 0.80, 1.1), "SPL_SP", "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["LepSF"] = input_df.Histo1D("GLepton_SF_nom")#, "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["LSF"] = input_df.Histo1D("wgt_LSF")#, "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["SPL_SP"] = input_df.Histo1D("SPL_SP")#, "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["SUMW_PU"] = input_df.Histo1D("wgt_SUMW_PU")#, "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["SUMW_PU_LSF"] = input_df.Histo1D("wgt_SUMW_PU_LSF")#, "wgt_SUMW_PU")
            #histos1D_dict["Leptons"]["PU"] = input_df.Histo1D("puWeight")#, "wgt_SUMW_PU")
    if doJets == True:
        if histos1D_dict != None:
            if "Jets" not in histos1D_dict:
                histos1D_dict["Jets"] = {}
            histos1D_dict["Jets"]["pt"] = input_df.Histo1D(("Jet_pt_({})".format(wgtVar), "", 100, 0, 500), "GJet_pt", wgtVar)
            for x in xrange(nJetsToHisto):
                histos1D_dict["Jets"]["pt_jet{}".format(x+1)] = input_df.Histo1D(("Jet_pt_jet{}({})".format(x+1, wgtVar), "", 100, 0, 500), "GJet_pt_jet{}".format(x+1), wgtVar)
            histos1D_dict["Jets"]["eta"] = input_df.Histo1D(("Jet_eta_({})".format(wgtVar), "", 104, -2.6, 2.6), "GJet_eta", wgtVar)
            histos1D_dict["Jets"]["phi"] = input_df.Histo1D(("Jet_phi_({})".format(wgtVar), "", 64, -pi, pi), "GJet_phi", wgtVar)
            histos1D_dict["Jets"]["mass"] = input_df.Histo1D(("Jet_mass_({})".format(wgtVar), "", 100, 0, 500), "GJet_mass", wgtVar)
            histos1D_dict["Jets"]["jetId"] = input_df.Histo1D(("Jet_jetId_({})".format(wgtVar), "", 8, 0, 8), "GJet_jetId", wgtVar)
            histos1D_dict["Jets"]["btagDeepB_LeadtagJet"] = input_df.Histo1D(("Jet_btagDeepB_LeadtagJet_({})".format(wgtVar), "", 101, -0.01, 1), "GJet_btagDeepB_LeadtagJet", wgtVar)
            histos1D_dict["Jets"]["btagDeepB_SubleadtagJet"] = input_df.Histo1D(("Jet_btagDeepB_SubleadtagJet_({})".format(wgtVar), "", 101, -0.01, 1), "GJet_btagDeepB_SubleadtagJet", wgtVar)
            histos1D_dict["Jets"]["btagDeepJet_LeadtagJet"] = input_df.Histo1D(("Jet_btagDeepJetB_LeadtagJet_({})".format(wgtVar), "", 101, -0.01, 1), "GJet_btagDeepFlavB_sorted_LeadtagJet", wgtVar)
            histos1D_dict["Jets"]["btagDeepJet_SubleadtagJet"] = input_df.Histo1D(("Jet_btagDeepJetB_SubleadtagJet_({})".format(wgtVar), "", 101, -0.01, 1), "GJet_btagDeepFlavB_sorted_SubleadtagJet", wgtVar)
            #histos1D_dict["Jets"]["nMediumCSVv2"] = input_df.Histo1D(("nJet_MediumCSVv2_({})".format(wgtVar), "", 10, 0, 10), "nGJet_MediumCSVv2", wgtVar)
            histos1D_dict["Jets"]["nMediumDeepCSV"] = input_df.Histo1D(("nJet_MediumDeepCSV_({})".format(wgtVar), "", 10, 0, 10), "nGJet_MediumDeepCSV", wgtVar)
            histos1D_dict["Jets"]["nMediumDeepJet"] = input_df.Histo1D(("nJet_MediumDeepJet_({})".format(wgtVar), "", 10, 0, 10), "nGJet_MediumDeepJet", wgtVar)
            histos1D_dict["Jets"]["nJet"] = input_df.Histo1D(("nJet_({})".format(wgtVar), "", 15, 0, 15), "nGJet", wgtVar)
            histos1D_dict["Jets"]["dR_Jet_Mu_leading"] = input_df.Histo1D(("dR_Jet_Mu_leading_({})".format(wgtVar), "dR(Jet, #mu_{leading}); dR; Events)", 40, 0, 0.8), "dR_Jet_Mu_leading", wgtVar)
            histos1D_dict["Jets"]["dR_Jet_Mu_sublead"] = input_df.Histo1D(("dR_Jet_Mu_sublead_({})".format(wgtVar), "dR(Jet, #mu_{subleading}); dR; Events)", 40, 0, 0.8), "dR_Jet_Mu_sublead", wgtVar)
            histos1D_dict["Jets"]["dR_Jet_El_leading"] = input_df.Histo1D(("dR_Jet_El_leading_({})".format(wgtVar), "dR(Jet, #e_{leading}); dR; Events)", 40, 0, 0.8), "dR_Jet_El_leading", wgtVar)
            histos1D_dict["Jets"]["dR_Jet_El_sublead"] = input_df.Histo1D(("dR_Jet_El_sublead_({})".format(wgtVar), "dR(Jet, #e_{subleading}); dR; Events)", 40, 0, 0.8), "dR_Jet_El_sublead", wgtVar)
                
            if debugInfo == True:
                #histos1D_dict["Jets"]["DiffMaskVsALT"] = input_df.Histo1D(("DiffMaskVsALT", "", 10, -10, 10), "DiffMaskVsALT", wgtVar)
                #histos1D_dict["Jets"]["DiffnJet"] = input_df.Histo1D(("DiffnJet", "", 10, -10, 10), "DiffnJet", wgtVar)
                histos1D_dict["Jets"]["DeepJetSorted"] = input_df.Histo1D("DeepJetSorted", wgtVar)
                histos1D_dict["Jets"]["DeepJetLeadtagMinusSubleadtag"] = input_df.Histo1D(("DeepJetLeadtagMinusSubleadtag", "DeepJet(Leadtag - Subleadtag);;Events", 100, -1, 1), "DeepJet0Minus1", wgtVar)
                histos1D_dict["Jets"]["MediumDeepJetSorted"] = input_df.Histo1D("MediumDeepJetSorted", wgtVar)
                #histos1D_dict["Jets"]["MediumDeepJet0Minus1"] = input_df.Histo1D(("MediumDeepJet0Minus1", "", 100, -1, 1), "MediumDeepJet0Minus1", wgtVar)
                #histos1D_dict["Jets"]["btagDeepJet_jet0Med"] = input_df.Histo1D(("Jet_btagDeepJetB_jet0Med_({})".format(wgtVar), "", 102, -0.02, 1), "GJet_btagDeepFlavB_jet0Med", wgtVar)
                #histos1D_dict["Jets"]["btagDeepJet_jet1Med"] = input_df.Histo1D(("Jet_btagDeepJetB_jet1Med_({})".format(wgtVar), "", 102, -0.02, 1), "GJet_btagDeepFlavB_jet1Med", wgtVar)
                #histos1D_dict["Jets"]["nJetNUMW"] = input_df.Histo1D(("nJet_NUMW", "", 15, 0, 15), "nGJet", "wgt_NUMW_V2")
                #histos1D_dict["Jets"]["nJetSUMW_PU"] = input_df.Histo1D(("nJet_SUMW_PU", "", 15, 0, 15), "nGJet", "wgt_SUMW_PU")
                #histos1D_dict["Jets"]["nJetSUMW_LSF"] = input_df.Histo1D(("nJet_SUMW_LSF", "", 15, 0, 15), "nGJet", "wgt_SUMW_LSF")
                #histos1D_dict["Jets"]["ptALT"] = input_df.Histo1D(("Jet_ptALT_({})".format(wgtVar), "", 100, 0, 500), "GJet_ptALT", wgtVar)
                #histos1D_dict["Jets"]["etaALT"] = input_df.Histo1D(("Jet_etaALT_({})".format(wgtVar), "", 104, -2.6, 2.6), "GJet_etaALT", wgtVar)
                #histos1D_dict["Jets"]["phiALT"] = input_df.Histo1D(("Jet_phiALT_({})".format(wgtVar), "", 64, -pi, pi), "GJet_phiALT", wgtVar)
                #histos1D_dict["Jets"]["massALT"] = input_df.Histo1D(("Jet_massALT_({})".format(wgtVar), "", 100, 0, 500), "GJet_massALT", wgtVar)
                #histos1D_dict["Jets"]["jetIdALT"] = input_df.Histo1D(("Jet_jetIdALT_({})".format(wgtVar), "", 8, 0, 8), "GJet_jetIdALT", wgtVar)
        if histos2D_dict != None:
            if "Jets" not in histos2D_dict:
                histos2D_dict["Jets"] = {}
            histos2D_dict["Jets"]["eta_phi"] = input_df.Histo2D(("Jet_eta_phi_({})".format(wgtVar), "",
                                                                 104, -2.6, 2.6,
                                                                 64, -pi, pi),
                                                                "GJet_eta", "GJet_phi", wgtVar)
    if doEventVars == True:
        if histos1D_dict != None:
            if "EventVars" not in histos1D_dict:
                histos1D_dict["EventVars"] = {}
            #histos1D_dict["EventVars"]["JML_baseline"] = input_df.Histo1D(("JML_baseline_({})".format(wgtVar), "", 2,0,2), "JML_baseline_pass", wgtVar)
            #histos1D_dict["EventVars"]["JML_selection"] = input_df.Histo1D(("JML_selection_({})".format(wgtVar), "", 2,0,2), "JML_selection_pass", wgtVar)
            #histos1D_dict["EventVars"]["HT_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_HT_baseline_({})".format(wgtVar), "", 100,400,1400), "ESV_JetMETLogic_HT_baseline", wgtVar)
            #histos1D_dict["EventVars"]["H_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_H_baseline_({})".format(wgtVar), "", 100,400,1400), "ESV_JetMETLogic_H_baseline", wgtVar)
            #histos1D_dict["EventVars"]["HT2M_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_HT2M_baseline_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_HT2M_baseline", wgtVar)
            #histos1D_dict["EventVars"]["H2M_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_H2M_baseline_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_H2M_baseline", wgtVar)
            #histos1D_dict["EventVars"]["HTb_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_HTb_baseline_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_HTb_baseline", wgtVar)
            #histos1D_dict["EventVars"]["HTH_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_HTH_baseline_({})".format(wgtVar), "", 100,0,1), "ESV_JetMETLogic_HTH_baseline", wgtVar)
            #histos1D_dict["EventVars"]["HTRat_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_HTRat_baseline_({})".format(wgtVar), "", 100,0,1), "ESV_JetMETLogic_HTRat_baseline", wgtVar)
            #histos1D_dict["EventVars"]["dRbb_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_dRbb_baseline_({})".format(wgtVar), "", 64,0,2*pi), "ESV_JetMETLogic_dRbb_baseline", wgtVar)
            #histos1D_dict["EventVars"]["DiLepMass_baseline"] = input_df.Histo1D(("ESV_JetMETLogic_DiLepMass_baseline_({})".format(wgtVar), "", 100,0,500), "ESV_JetMETLogic_DiLepMass_baseline", wgtVar)
            #histos1D_dict["EventVars"]["HT_selection"] = input_df.Histo1D(("ESV_JetMETLogic_HT_selection_({})".format(wgtVar), "", 100,400,1400), "ESV_JetMETLogic_HT_selection", wgtVar)
            #histos1D_dict["EventVars"]["H_selection"] = input_df.Histo1D(("ESV_JetMETLogic_H_selection_({})".format(wgtVar), "", 100,400,1400), "ESV_JetMETLogic_H_selection", wgtVar)
            #histos1D_dict["EventVars"]["HT2M_selection"] = input_df.Histo1D(("ESV_JetMETLogic_HT2M_selection_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_HT2M_selection", wgtVar)
            #histos1D_dict["EventVars"]["H2M_selection"] = input_df.Histo1D(("ESV_JetMETLogic_H2M_selection_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_H2M_selection", wgtVar)
            #histos1D_dict["EventVars"]["HTb_selection"] = input_df.Histo1D(("ESV_JetMETLogic_HTb_selection_({})".format(wgtVar), "", 100,400,900), "ESV_JetMETLogic_HTb_selection", wgtVar)
            #histos1D_dict["EventVars"]["HTH_selection"] = input_df.Histo1D(("ESV_JetMETLogic_HTH_selection_({})".format(wgtVar), "", 100,0,1), "ESV_JetMETLogic_HTH_selection", wgtVar)
            #histos1D_dict["EventVars"]["HTRat_selection"] = input_df.Histo1D(("ESV_JetMETLogic_HTRat_selection_({})".format(wgtVar), "", 100,0,1), "ESV_JetMETLogic_HTRat_selection", wgtVar)
            #histos1D_dict["EventVars"]["dRbb_selection"] = input_df.Histo1D(("ESV_JetMETLogic_dRbb_selection_({})".format(wgtVar), "", 64,0,2*pi), "ESV_JetMETLogic_dRbb_selection", wgtVar)
            #histos1D_dict["EventVars"]["DiLepMass_selection"] = input_df.Histo1D(("ESV_JetMETLogic_DiLepMass_selection_({})".format(wgtVar), "", 100,0,500), "ESV_JetMETLogic_DiLepMass_selection", wgtVar)
            histos1D_dict["EventVars"]["MET_pt"] = input_df.Histo1D(("MET_pt_({})".format(wgtVar), "", 100,30,1030), "METFixEE2017_pt", wgtVar)
            histos1D_dict["EventVars"]["MET_phi"] = input_df.Histo1D(("MET_phi_({})".format(wgtVar), "", 100,-pi,pi), "METFixEE2017_phi", wgtVar)
            histos1D_dict["EventVars"]["HT"] = input_df.Histo1D(("HT_({})".format(wgtVar), "", 130,400,1700), "GJet_HT", wgtVar)
            histos1D_dict["EventVars"]["H"] = input_df.Histo1D(("H_({})".format(wgtVar), "", 160,400,2000), "GJet_H", wgtVar)
            histos1D_dict["EventVars"]["HT2M"] = input_df.Histo1D(("HT2M_({})".format(wgtVar), "", 100,0,1000), "GJet_HT2M", wgtVar)
            histos1D_dict["EventVars"]["H2M"] = input_df.Histo1D(("H2M_({})".format(wgtVar), "", 150,0,1500), "GJet_H2M", wgtVar)
            histos1D_dict["EventVars"]["HTb"] = input_df.Histo1D(("HTb_({})".format(wgtVar), "", 100,0,1000), "GJet_HTb", wgtVar)
            histos1D_dict["EventVars"]["HTH"] = input_df.Histo1D(("HTH_({})".format(wgtVar), "", 100,0,1), "GJet_HTH", wgtVar)
            histos1D_dict["EventVars"]["HTRat"] = input_df.Histo1D(("HTRat_({})".format(wgtVar), "", 100,0,1), "GJet_HTRat", wgtVar)
            histos1D_dict["EventVars"]["dRbb"] = input_df.Histo1D(("dRbb_({})".format(wgtVar), "", 64,0,2*pi), "GJet_dRbb", wgtVar)
            histos1D_dict["EventVars"]["dPhibb"] = input_df.Histo1D(("dPhibb_({})".format(wgtVar), "", 64,-pi,pi), "GJet_dPhibb", wgtVar)
            histos1D_dict["EventVars"]["dEtabb"] = input_df.Histo1D(("dEtabb_({})".format(wgtVar), "", 50,0,5), "GJet_dEtabb", wgtVar)
            histos1D_dict["EventVars"]["dRll"] = input_df.Histo1D(("dRll_({})".format(wgtVar), "", 64,0,2*pi), "GLepton_dRll", wgtVar)
            histos1D_dict["EventVars"]["dPhill"] = input_df.Histo1D(("dPhill_({})".format(wgtVar), "", 64,-pi,pi), "GLepton_dPhill", wgtVar)
            histos1D_dict["EventVars"]["dEtall"] = input_df.Histo1D(("dEtall_({})".format(wgtVar), "", 50,0,5), "GLepton_dEtall", wgtVar)
            histos1D_dict["EventVars"]["MTofMETandEl"] = input_df.Histo1D(("MTofMETandEl_({})".format(wgtVar), "", 100, 0, 200), "MTofMETandEl", wgtVar)
            histos1D_dict["EventVars"]["MTofMETandMu"] = input_df.Histo1D(("MTofMETandMu_({})".format(wgtVar), "", 100, 0, 200), "MTofMETandMu", wgtVar)
            histos1D_dict["EventVars"]["MTofElandMu"] = input_df.Histo1D(("MTofElandMu_({})".format(wgtVar), "", 100, 0, 200), "MTofElandMu", wgtVar)
            #histos1D_dict["EventVars"]["MTMasslessCheck"] = input_df.Histo1D(("MTMasslessCheck_({})".format(wgtVar), "", 100, 0, 200), "MTMasslessCheck", wgtVar)
            #histos1D_dict["EventVars"]["MTCrossCheck"] = input_df.Histo1D(("MTCrossCheck_({})".format(wgtVar), "", 100, 0, 200), "MTCrossCheck", wgtVar)
            #histos1D_dict["EventVars"]["MTCrossCheckDifference"] = input_df.Histo1D(("MTCrossCheckDifference_({})".format(wgtVar), "", 100, 0, 10), "MTCrossCheckDifference", wgtVar)
            #histos1D_dict["EventVars"]["MTCrossCheckMasslessDifference"] = input_df.Histo1D(("MTCrossCheckMasslessDifference_({})".format(wgtVar), "", 100, 0, 0.02), "MTCrossCheckMasslessDifference", wgtVar)
            histos1D_dict["EventVars"]["PV_npvsGood"] = input_df.Histo1D(("PV_npvsGood_({})".format(wgtVar), "", 100, 0, 100), "PV_npvsGood", wgtVar)
            histos1D_dict["EventVars"]["PV_npvs"] = input_df.Histo1D(("PV_npvs_({})".format(wgtVar), "", 150, 0, 150), "PV_npvs", wgtVar)
            if isData == False:
                histos1D_dict["EventVars"]["Pileup_nTrueInt"] = input_df.Histo1D(("Pileup_TrueInt_({})".format(wgtVar), ";Pileup_TrueInt;Events", 150, 0, 150), "Pileup_nTrueInt", wgtVar)
                histos1D_dict["EventVars"]["Pileup_nTrueInt_XS"] = input_df.Histo1D(("Pileup_TrueInt_({})".format("wgt_SUMW"), ";Pileup_TrueInt;Events", 150, 0, 150), "Pileup_nTrueInt", "wgt_SUMW")
                histos1D_dict["EventVars"]["Pileup_nPU_XS"] = input_df.Histo1D(("Pileup_nPU_({})".format("wgt_SUMW"), ";Pileup_nPU;Events", 150, 0, 150), "Pileup_nPU", "wgt_SUMW")
                histos1D_dict["EventVars"]["Pileup_nPU"] = input_df.Histo1D(("Pileup_nPU_({})".format(wgtVar), ";Pileup_nPU;Events", 150, 0, 150), "Pileup_nPU", wgtVar)
            
        if histos2D_dict != None:
            if "EventVars" not in histos2D_dict:
                histos2D_dict["EventVars"] = {}
            if isData == False:
                histos2D_dict["EventVars"]["npvsGood_vs_nTrueInt"] = input_df.Histo2D(("npvsGood_vs_nTrueInt_({})".format(wgtVar), ";nTrueInt;npvsGood", 150, 0, 150, 150, 0, 150), "Pileup_nTrueInt", "PV_npvsGood", wgtVar)
                histos2D_dict["EventVars"]["npvsGood_vs_nPU"] = input_df.Histo2D(("npvsGood_vs_nPU_({})".format(wgtVar), ";nPU;npvsGood", 150, 0, 150, 150, 0, 150), "Pileup_nPU", "PV_npvsGood", wgtVar)
                histos2D_dict["EventVars"]["npvs_vs_nTrueInt"] = input_df.Histo2D(("npvs_vs_nTrueInt_({})".format(wgtVar), ";nTrueInt;npvs", 150, 0, 150, 150, 0, 150), "Pileup_nTrueInt", "PV_npvs", wgtVar)
                histos2D_dict["EventVars"]["npvs_vs_nPU"] = input_df.Histo2D(("npvs_vs_nPU_({})".format(wgtVar), ";nPU;npvs", 150, 0, 150, 150, 0, 150), "Pileup_nPU", "PV_npvs", wgtVar)
            
            if debugInfo == True:
                #histos1D_dict["EventVars"]["GJet_HT_Match"] = input_df.Histo1D(("GJet_HT_Match_({})".format(wgtVar), "", 2,0,2), "GJet_HT_matches", wgtVar)
                pass
            
    if makeMountains == True:
        theCats = collections.OrderedDict()
        theCats["nJet4"] = "nGJet == 4"
        theCats["nJet5"] = "nGJet == 5"
        theCats["blind_nJet6"] = "nGJet == 6"
        theCats["blind_nJet7"] = "nGJet == 7"
        theCats["blind_nJet8+"] = "nGJet >= 8"
        #theCats["nMediumDeepCSV0"] = "nGJet_MediumDeepCSV == 0"
        #theCats["nMediumDeepCSV1"] = "nGJet_MediumDeepCSV == 1"
        theCats["nMediumDeepJet0"] = "nGJet_MediumDeepJet == 0"
        theCats["nMediumDeepJet1"] = "nGJet_MediumDeepJet == 1"
        theCats["nMediumDeepJet2"] = "nGJet_MediumDeepJet == 2"
        
        theCats["nMediumDeepJet0_nJet4"] = "nGJet_MediumDeepJet == 0 && nGJet == 4"
        theCats["nMediumDeepJet1_nJet4"] = "nGJet_MediumDeepJet == 1 && nGJet == 4"
        theCats["nMediumDeepJet2_nJet4"] = "nGJet_MediumDeepJet == 2 && nGJet == 4"
        theCats["blind_nMediumDeepJet3_nJet4"] = "nGJet_MediumDeepJet == 3 && nGJet == 4"
        theCats["blind_nMediumDeepJet4+_nJet4"] = "nGJet_MediumDeepJet >= 4 && nGJet == 4"
        
        theCats["nMediumDeepJet0_nJet5"] = "nGJet_MediumDeepJet == 0 && nGJet == 5"
        theCats["nMediumDeepJet1_nJet5"] = "nGJet_MediumDeepJet == 1 && nGJet == 5"
        theCats["nMediumDeepJet2_nJet5"] = "nGJet_MediumDeepJet == 2 && nGJet == 5"
        theCats["blind_nMediumDeepJet3_nJet5"] = "nGJet_MediumDeepJet == 3 && nGJet == 5"
        theCats["blind_nMediumDeepJet4+_nJet5"] = "nGJet_MediumDeepJet >= 4 && nGJet == 5"
        
        theCats["nMediumDeepJet0_nJet6"] = "nGJet_MediumDeepJet == 0 && nGJet == 6"
        theCats["nMediumDeepJet1_nJet6"] = "nGJet_MediumDeepJet == 1 && nGJet == 6"
        theCats["nMediumDeepJet2_nJet6"] = "nGJet_MediumDeepJet == 2 && nGJet == 6"
        theCats["blind_nMediumDeepJet3_nJet6"] = "nGJet_MediumDeepJet == 3 && nGJet == 6"
        theCats["blind_nMediumDeepJet4+_nJet6"] = "nGJet_MediumDeepJet >= 4 && nGJet == 6"
        
        theCats["nMediumDeepJet0_nJet7"] = "nGJet_MediumDeepJet == 0 && nGJet == 7"
        theCats["nMediumDeepJet1_nJet7"] = "nGJet_MediumDeepJet == 1 && nGJet == 7"
        theCats["blind_nMediumDeepJet2_nJet7"] = "nGJet_MediumDeepJet == 2 && nGJet == 7"
        theCats["blind_nMediumDeepJet3_nJet7"] = "nGJet_MediumDeepJet == 3 && nGJet == 7"
        theCats["blind_nMediumDeepJet4+_nJet7"] = "nGJet_MediumDeepJet >= 4 && nGJet == 7"
        
        theCats["nMediumDeepJet0_nJet8+"] = "nGJet_MediumDeepJet == 0 && nGJet >= 8"
        theCats["nMediumDeepJet1_nJet8+"] = "nGJet_MediumDeepJet == 1 && nGJet >= 8"
        theCats["blind_nMediumDeepJet2_nJet8+"] = "nGJet_MediumDeepJet == 2 && nGJet >= 8"
        theCats["blind_nMediumDeepJet3_nJet8+"] = "nGJet_MediumDeepJet == 3 && nGJet >= 8"
        theCats["blind_nMediumDeepJet4+_nJet8+"] = "nGJet_MediumDeepJet >= 4 && nGJet >= 8"
        cat_df = collections.OrderedDict()
        for ck, cs in theCats.items():
            cat_df[ck] = input_df.Filter(cs, cs)
        if histos1D_dict != None:
            if "Mountains" not in histos1D_dict:
                histos1D_dict["Mountains"] = {}
            for tc in theCats.keys(): 
                if tc not in histos1D_dict["Mountains"]: 
                    histos1D_dict["Mountains"][tc] = {}
            for tc, cut in theCats.items():
                tcn = tc.replace("blind_", "")
                histos1D_dict["Mountains"][tc]["MET_pt"] = cat_df[tc].Histo1D(("MET_pt_[{}]({})".format(tcn, wgtVar), "", 20,30,1030), "METFixEE2017_pt", wgtVar)
                histos1D_dict["Mountains"][tc]["MET_phi"] = cat_df[tc].Histo1D(("MET_phi_[{}]({})".format(tcn, wgtVar), "", 20,-pi,pi), "METFixEE2017_phi", wgtVar)
                histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_all"] = cat_df[tc].Histo1D(("Muon_pfRelIso03_all_[{}]({})".format(tcn, wgtVar), "", 20, 0, 0.2), "GMuon_pfRelIso03_all", wgtVar)
                histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_chg"] = cat_df[tc].Histo1D(("Muon_pfRelIso03_chg_[{}]({})".format(tcn, wgtVar), "", 20, 0, 0.2), "GMuon_pfRelIso03_chg", wgtVar)
                histos1D_dict["Mountains"][tc]["Muon_pfRelIso04_all"] = cat_df[tc].Histo1D(("Muon_pfRelIso04_all_[{}]({})".format(tcn, wgtVar), "", 20, 0, 0.2), "GMuon_pfRelIso04_all", wgtVar)
                histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_all"] = cat_df[tc].Histo1D(("Electron_pfRelIso03_all_[{}]({})".format(tcn, wgtVar), "", 20, 0, 0.2), "GElectron_pfRelIso03_all", wgtVar)
                histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_chg"] = cat_df[tc].Histo1D(("Electron_pfRelIso03_chg_[{}]({})".format(tcn, wgtVar), "", 20, 0, 0.2), "GElectron_pfRelIso03_chg", wgtVar)
                histos1D_dict["Mountains"][tc]["HT"] = cat_df[tc].Histo1D(("HT_[{}]({})".format(tcn, wgtVar), "", 30,400,2000), "GJet_HT", wgtVar)
                histos1D_dict["Mountains"][tc]["H"] = cat_df[tc].Histo1D(("H_[{}]({})".format(tcn, wgtVar), "", 30,400,2000), "GJet_H", wgtVar)
                histos1D_dict["Mountains"][tc]["HT2M"] = cat_df[tc].Histo1D(("HT2M_[{}]({})".format(tcn, wgtVar), "", 20,0,1000), "GJet_HT2M", wgtVar)
                histos1D_dict["Mountains"][tc]["H2M"] = cat_df[tc].Histo1D(("H2M_[{}]({})".format(tcn, wgtVar), "", 20,0,1500), "GJet_H2M", wgtVar)
                histos1D_dict["Mountains"][tc]["HTb"] = cat_df[tc].Histo1D(("HTb_[{}]({})".format(tcn, wgtVar), "", 20,0,1000), "GJet_HTb", wgtVar)
                histos1D_dict["Mountains"][tc]["HTH"] = cat_df[tc].Histo1D(("HTH_[{}]({})".format(tcn, wgtVar), "", 20,0,1), "GJet_HTH", wgtVar)
                histos1D_dict["Mountains"][tc]["HTRat"] = cat_df[tc].Histo1D(("HTRat_[{}]({})".format(tcn, wgtVar), "", 20,0,1), "GJet_HTRat", wgtVar)
                histos1D_dict["Mountains"][tc]["dRbb"] = cat_df[tc].Histo1D(("dRbb_[{}]({})".format(tcn, wgtVar), "", 16,0,2*pi), "GJet_dRbb", wgtVar)
                histos1D_dict["Mountains"][tc]["dPhibb"] = cat_df[tc].Histo1D(("dPhibb_[{}]({})".format(tcn, wgtVar), "", 16,-pi,pi), "GJet_dPhibb", wgtVar)
                histos1D_dict["Mountains"][tc]["dEtabb"] = cat_df[tc].Histo1D(("dEtabb_[{}]({})".format(tcn, wgtVar), "", 10,0,5), "GJet_dEtabb", wgtVar)
                histos1D_dict["Mountains"][tc]["dRll"] = cat_df[tc].Histo1D(("dRll_[{}]({})".format(tcn, wgtVar), "", 16,0,2*pi), "GLepton_dRll", wgtVar)
                histos1D_dict["Mountains"][tc]["dPhill"] = cat_df[tc].Histo1D(("dPhill_[{}]({})".format(tcn, wgtVar), "", 16,-pi,pi), "GLepton_dPhill", wgtVar)
                histos1D_dict["Mountains"][tc]["dEtall"] = cat_df[tc].Histo1D(("dEtall_[{}]({})".format(tcn, wgtVar), "", 10,0,5), "GLepton_dEtall", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofMETandEl"] = cat_df[tc].Histo1D(("MTofMETandEl_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofMETandEl", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofMETandMu"] = cat_df[tc].Histo1D(("MTofMETandMu_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofMETandMu", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofElandMu"] = cat_df[tc].Histo1D(("MTofElandMu_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofElandMu", wgtVar)
                histos1D_dict["Mountains"][tc]["nMediumDeepCSV"] = cat_df[tc].Histo1D(("nJet_MediumDeepCSV_[{}]({})".format(tcn, wgtVar), "", 10, 0, 10), "nGJet_MediumDeepCSV", wgtVar)
                histos1D_dict["Mountains"][tc]["nMediumDeepJet"] = cat_df[tc].Histo1D(("nJet_MediumDeepJet_[{}]({})".format(tcn, wgtVar), "", 10, 0, 10), "nGJet_MediumDeepJet", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofMETandEl"] = cat_df[tc].Histo1D(("MTofMETandEl_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofMETandEl", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofMETandMu"] = cat_df[tc].Histo1D(("MTofMETandMu_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofMETandMu", wgtVar)
                histos1D_dict["Mountains"][tc]["MTofElandMu"] = cat_df[tc].Histo1D(("MTofElandMu_[{}]({})".format(tcn, wgtVar), "", 20, 0, 200), "MTofElandMu", wgtVar)
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 30, 0., 0.2, 20,30.,1030.), "GMuon_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 30, 0, 0.2, 20,30,1030), "GMuon_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso04_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso04_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso04_all;MET", 30, 0, 0.2, 20,30,1030), "GMuon_pfRelIso04_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 30, 0, 0.2, 20,30,1030), "GElectron_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 30, 0, 0.2, 20,30,1030), "GElectron_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                
                if isData == False:
                    pass                
            
        if histos2D_dict != None:
            if "Mountains" not in histos2D_dict:
                histos2D_dict["Mountains"] = {}
            for tc in theCats.keys(): 
                if tc not in histos2D_dict["Mountains"]: 
                    histos2D_dict["Mountains"][tc] = {}
            for tc, cut in theCats.items():
                tcn = tc.replace("blind_", "")
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 30, 0., 0.2, 20,30.,1030.), "GMuon_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 30, 0, 0.2, 20,30,1030), "GMuon_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Muon_pfRelIso04_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso04_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso04_all;MET", 30, 0, 0.2, 20,30,1030), "GMuon_pfRelIso04_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 30, 0, 0.2, 20,30,1030), "GElectron_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos1D_dict["Mountains"][tc]["Electron_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 30, 0, 0.2, 20,30,1030), "GElectron_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                #### Older versions
                #histos2D_dict["Mountains"][tc]["Muon_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 100, 0., 0.2, 100,30.,1030.), "GMuon_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos2D_dict["Mountains"][tc]["Muon_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 100, 0, 0.2, 100,30,1030), "GMuon_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                #histos2D_dict["Mountains"][tc]["Muon_pfRelIso04_all_vs_MET"] = cat_df[tc].Histo2D(("Muon_pfRelIso04_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso04_all;MET", 100, 0, 0.2, 100,30,1030), "GMuon_pfRelIso04_all", "RVec_MET_pt", wgtVar)
                #histos2D_dict["Mountains"][tc]["Electron_pfRelIso03_all_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_all_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_all;MET", 100, 0, 0.2, 100,30,1030), "GElectron_pfRelIso03_all", "RVec_MET_pt", wgtVar)
                #histos2D_dict["Mountains"][tc]["Electron_pfRelIso03_chg_vs_MET"] = cat_df[tc].Histo2D(("Electron_pfRelIso03_chg_vs_MET_[{}]({})".format(tcn, wgtVar), ";pfRelIso03_chg;MET", 100, 0, 0.2, 100,30,1030), "GElectron_pfRelIso03_chg", "RVec_MET_pt", wgtVar)
                if isData == False:
                    #histos2D_dict["Mountains"][tc]["test1"] = cat_df[tc].Histo2D(("test1_[{}]({})".format(tcn, wgtVar), ";nTrueInt;npvsGood", 150, 0, 150, 150, 0, 150), "GMuon_pfRelIso03_all", "PV_npvsGood", wgtVar)
                    #histos2D_dict["Mountains"][tc]["test2"] = cat_df[tc].Histo2D(("test2_[{}]({})".format(tcn, wgtVar), ";nTrueInt;npvsGood", 150, 0, 150, 150, 0, 150), "GMuon_pfRelIso03_all", "METFixEE2017_pt", wgtVar)
                    #histos2D_dict["Mountains"][tc]["npvsGood_vs_nTrueInttest"] = cat_df[tc].Histo2D(("npvsGood_vs_nTrueInttest_[{}]({})".format(tcn, wgtVar), ";nTrueInt;npvsGood", 150, 0, 150, 150, 0, 150), "GElectron_pfRelIso03_all", "MET_pt_flat", wgtVar)
                    histos2D_dict["Mountains"][tc]["npvsGood_vs_nTrueInt"] = cat_df[tc].Histo2D(("npvsGood_vs_nTrueInt_[{}]({})".format(tcn, wgtVar), ";nTrueInt;npvsGood", 150, 0, 150, 150, 0, 150), "Pileup_nTrueInt", "PV_npvsGood", wgtVar)
                    histos2D_dict["Mountains"][tc]["npvsGood_vs_nPU"] = cat_df[tc].Histo2D(("npvsGood_vs_nPU_[{}]({})".format(tcn, wgtVar), ";nPU;npvsGood", 150, 0, 150, 150, 0, 150), "Pileup_nPU", "PV_npvsGood", wgtVar)
                    histos2D_dict["Mountains"][tc]["npvs_vs_nTrueInt"] = cat_df[tc].Histo2D(("npvs_vs_nTrueInt_[{}]({})".format(tcn, wgtVar), ";nTrueInt;npvs", 150, 0, 150, 150, 0, 150), "Pileup_nTrueInt", "PV_npvs", wgtVar)
                    histos2D_dict["Mountains"][tc]["npvs_vs_nPU"] = cat_df[tc].Histo2D(("npvs_vs_nPU_[{}]({})".format(tcn, wgtVar), ";nPU;npvs", 150, 0, 150, 150, 0, 150), "Pileup_nPU", "PV_npvs", wgtVar)
      
            

In [15]:
#Jet bit dictionary, for reference
JMLpassbits = {'PV_minNDoF':                            0b00000000000000000001,
                 'PV_maxAbsZ':                            0b00000000000000000010,
                 'PV_maxRho':                             0b00000000000000000100,
                 'MET_globalSuperTightHalo2016Filter':    0b00000000000000001000,
                 'MET_goodVertices':                      0b00000000000000010000,
                 'MET_HBHENoiseFilter':                   0b00000000000000100000,
                 'MET_HBHENoiseIsoFilter':                0b00000000000001000000,
                 'MET_EcalDeadCellTriggerPrimitiveFilter':0b00000000000010000000,
                 'MET_BadPFMuonFilter':                   0b00000000000100000000,
                 'MET_ecalBadCalibFilterV2':              0b00000000001000000000,
                 'MET_pt':                                0b00000000010000000000,
                 'unused1':                               0b00000000100000000000, #N
                 'Lepton_ZWindow':                        0b00000001000000000000, #N
                 'Jet_nJet25':                            0b00000010000000000000, #N
                 'Jet_nJet20':                            0b00000100000000000000,
                 'HT':                                    0b00001000000000000000,
                 'Jet_nBJet_2DCSV':                       0b00010000000000000000, #N
                 'Jet_nBJet_2DJet':                       0b00100000000000000000, #N
                 'unused2':                               0b01000000000000000000, #N
                 'unused3':                               0b10000000000000000000, #N
                }

In [16]:
##################################################
##################################################
### CHOOSE SAMPLE DICT AND CHANNEL TO ANALYZE ####
##################################################
##################################################

#Focus on limited set of events at a time
levels_of_interest = set(["ElMu_selection"])
#levels_of_interest = set(["selection", "ElMu_selection", "ElEl_selection", "MuMu_selection", "Mu_selection", "El_selection"])
#levels_of_interest = set(["baseline", "MuMu_baseline", "ElEl_baseline", "selection", "MuMu_selection", "ElMu_selection"])
#levels_of_interest = set(["baseline", "MuMu_selection", "ElMu_selection"])

#Choose the sample dictionary to run
#theSampleDict = ttbooker #needs modification to make work...
#theSampleDict = ttttbooker
theSampleDict = microbooker #tttt, ttbar-DL unfiltered, DY, one single top sample
#theSampleDict = minibooker #tttt, all ttbar, both single top, DY
#theSampleDict = booker #All
#theSampleDict = bookerV2 #All with reprocessing (WIP: Other data streams, ttVJets, Filtered samples!)
#theSampleDict = tt_data_V2
#theSampleDict = pyrdfbooker

#Choose the weight variation
#theWeight = "wgt_SUMW"
#theWeight = "wgt_SUMW_PU"
#theWeight = "wgt_SUMW_LSF"
#theWeight = "wgt_SUMW_L1PF"
#theWeight = "wgt_SUMW_PU_LSF"
theWeight = "wgt_SUMW_PU_LSF_L1PF"
#theWeight = "wgt_SUMW_LSF_L1PF"
#theWeight = "wgt_NUMW_LSF_L1PF"

#Name the channel that's being analyzed for saving files, and the format (.C, .root, .pdf, .eps, .gif, .png, .jpeg, etc)
fileChannel = "ElMu"
#theFormat = ".pdf"
theFormat = ".png"

In [17]:
print("Creating selection and baseline bits")
b = {}
b["ElMu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) > 0".format(Chan["ElMu_baseline"])
b["MuMu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"], 
                                                                                                                                Chan["MuMu_baseline"])
b["ElEl_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"], 
                                                                                                                                Chan["ElEl_baseline"])
b["Mu_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + Chan["ElEl_baseline"], Chan["Mu_baseline"])
b["El_baseline"] = "(ESV_TriggerAndLeptonLogic_baseline & {0}) == 0 && (ESV_TriggerAndLeptonLogic_baseline & {1}) > 0".format(Chan["ElMu_baseline"] + Chan["MuMu_baseline"] + 
                                                                                                                    Chan["ElEl_baseline"] + Chan["Mu_baseline"], Chan["El_baseline"])
b["selection"] = "ESV_TriggerAndLeptonLogic_selection > 0"
b["ElMu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) > 0".format(Chan["ElMu_selection"])
b["MuMu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"], Chan["MuMu_selection"])
b["ElEl_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"], Chan["ElEl_selection"])
b["Mu_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"], Chan["Mu_selection"])
b["El_selection"] = "(ESV_TriggerAndLeptonLogic_selection & {0}) == 0 && (ESV_TriggerAndLeptonLogic_selection & {1}) > 0".format(Chan["ElMu_selection"] + Chan["MuMu_selection"] + Chan["ElEl_selection"]
                                                                            + Chan["Mu_selection"], Chan["El_selection"]) 
b["ESV_JetMETLogic_baseline"] = "(ESV_JetMETLogic_baseline & {0}) >= {0}".format(0b00001100011111111111)
#b["ESV_JetMETLogic_selection"] = "(ESV_JetMETLogic_baseline & {0}) >= {0}".format(0b00001100011111111111) #FIXME, this isn't right!
b["ESV_JetMETLogic_selection"] = "(ESV_JetMETLogic_selection & {0}) >= {0}".format(0b00001100011111111111)
b["ESV_JetMETLogic_default"] = "(ESV_JetMETLogic_baseline & {}) > 0".format(0b11111111111111111111)
#print(b["ESV_JetMETLogic_selection"])



filtered = {}
for name, vals in theSampleDict.items():
    if name == "tttt_orig": continue
    print("Initializing RDataFrame - {}".format(name))
    filtered[name] = {}
    for lvl in levels_of_interest:
        if "baseline" in lvl:
            JMLOG = "ESV_JetMETLogic_baseline"
        elif "selection" in lvl:
            JMLOG = "ESV_JetMETLogic_selection"
        else:
            JMLOG = "ESV_JetMETLogic_default"
        if lvl == "baseline":
            filtered[name][lvl] = RDF("Events", vals["source"])#.Filter(b[JMLOG], JMLOG)#.Cache()
        else:
            filtered[name][lvl] = RDF("Events", vals["source"])#.Filter(b[lvl], lvl).Filter(b[JMLOG], JMLOG)#.Cache()
        #Cache() seemingly has an issue with the depth/breadth of full NanoAOD file. Perhaps one with fewer branches would work
        #filtered[name][lvl] = filtered[name][lvl].Cache()

Creating selection and baseline bits
Initializing RDataFrame - tt_DL
Initializing RDataFrame - tttt
Initializing RDataFrame - ST_tW


In [18]:
samples = {}
counts = {}
histos1D = {}
histos1D_PU = {}
histos2D = {}
histosNS = {} #unstacked histograms
the_df = {}
print("Starting loop for booking")
for name, vals in theSampleDict.items():
    if name == "tttt_orig": continue
    #if name not in ["tttt", "ElMu_F"]: continue
    print("Booking - {}".format(name))
    counts[name] = {}
    histos1D[name] = {}
    histos1D_PU[name] = {}
    histos2D[name] = {}
    histosNS[name] = {}
    the_df[name] = {}
    #counts[name]["baseline"] = filtered[name].Count() #Unnecessary with baseline in levels of interest?
    for lvl in levels_of_interest:
        the_df[name][lvl] = defineLeptons(filtered[name][lvl], 
                                            input_lvl_filter=lvl, 
                                            channel="MuMu", 
                                            isData=vals["isData"], 
                                            useBackupChannel=False)
        if vals["isData"] == False:
            the_df[name][lvl] = defineWeights(the_df[name][lvl],
                                            crossSection=vals["crossSection"], 
                                            sumWeights=vals["sumWeights"], 
                                            lumi=lumi[era],
                                            nEvents=vals["nEvents"], 
                                            nEventsPositive=vals["nEventsPositive"], 
                                            nEventsNegative=vals["nEventsNegative"],
                                            channel="MuMu", 
                                            isData=vals["isData"], 
                                            verbose=False,)
        else:
            the_df[name][lvl] = defineWeights(the_df[name][lvl],
                                             isData=True,
                                             verbose=False)
        the_df[name][lvl] = defineJets(the_df[name][lvl],
                                       era="2017",
                                      )
        the_df[name][lvl] = defineEventVars(the_df[name][lvl])
        the_df[name][lvl] = the_df[name][lvl].Define("RVec_MET_pt", "ROOT::VecOps::RVec<float> encaps; encaps.push_back(METFixEE2017_pt); return encaps;")
        #the_df[name][lvl] = the_df[name][lvl].Filter("nGJet > 2", "nJet > 2")
        the_df[name][lvl] = the_df[name][lvl].Filter("nGJet > 3", "nJet > 3")
        #the_df[name][lvl] = the_df[name][lvl].Filter("nGJet_MediumDeepCSV > 1")
        #the_df[name][lvl] = the_df[name][lvl].Filter("nGJet_MediumDeepJet > 1", "nMedDeepJet > 1")
        #the_df[name][lvl] = the_df[name][lvl].Filter("METFixEE2017_pt > 40", "MET > 40")
        the_df[name][lvl] = the_df[name][lvl].Filter("METFixEE2017_pt > 50", "MET > 50")
        #the_df[name][lvl] = the_df[name][lvl].Filter("GJet_HT > 450", "HT > 450")
        the_df[name][lvl] = the_df[name][lvl].Filter("GJet_HT > 500", "HT > 500")
        counts[name][lvl] = the_df[name][lvl].Count()
        histos1D[name][lvl] = {}
        histos1D_PU[name][lvl] = {}
        histosNS[name][lvl] = {}
        histos2D[name][lvl] = {}
        fillHistos(the_df[name][lvl], wgtVar=theWeight, isData = vals["isData"],
                   histos1D_dict=histos1D[name][lvl], histos2D_dict=histos2D[name][lvl], 
                   histosNS_dict=histosNS[name][lvl],
                   doMuons=True, doElectrons=True, doLeptons=True, 
                   doJets=True, doWeights=False, doEventVars=True,
                   makeMountains=True)
#        fillHistos(the_df[name][lvl], wgtVar="wgt_SUMW_PU_LSF", histos1D_dict=histos1D[name][lvl], 
#                   histos2D_dict=histos2D[name][lvl], histosNS_dict=histosNS[name][lvl],
#                   doMuons=False, doElectrons=False, doLeptons=True, 
#                   doJets=False, doWeights=True, doEventVars=False)
#        fillHistos(the_df[name][lvl], wgtVar="wgt_SUMW_PU", histos1D_dict=histos1D[name][lvl], 
#                   histos2D_dict=histos2D[name][lvl], histosNS_dict=histosNS[name][lvl],
#                   doMuons=False, doElectrons=False, doLeptons=False, 
#                   doJets=False, doWeights=False, doEventVars=True)
#        fillHistos(the_df[name][lvl], wgtVar="wgt_SUMW_PU", histos1D_dict=histos1D_PU[name][lvl], 
#                   histos2D_dict=histos2D[name][lvl], histosNS_dict=histosNS[name][lvl],
#                   doMuons=True, doElectrons=True, doLeptons=True, 
#                   doJets=True, doWeights=True, doEventVars=False)

Starting loop for booking
Booking - tt_DL
FIXMEFIXME: Setting Jet_pt min to 30GeV! Must fix!
Booking - tttt
FIXMEFIXME: Setting Jet_pt min to 30GeV! Must fix!
Booking - ST_tW
FIXMEFIXME: Setting Jet_pt min to 30GeV! Must fix!


In [19]:
print("Warning: if filtered[name][lvl] RDFs are not reset, then calling Define(*) on them will cause the error"\
      " with 'program state reset' due to multiple definitions for the same variable")
loopcounter = 0
start = time.clock()
substart = {}
subfinish = {}
for name, cnt in counts.items():
    substart[name] = time.clock()
    loopcounter += 1
    print("==========={}/{}\n{}".format(loopcounter, len(counts), name))
    if "baseline" in cnt:
        print("Baseline = " + str(cnt["baseline"].GetValue()))
    else:
        print("Baseline")
    if "ElMu_baseline" in cnt:
        print("\tElMu = {}".format(cnt["ElMu_baseline"].GetValue()),end='')
    if "MuMu_baseline" in cnt:
        print("\tMuMu = {}".format(cnt["MuMu_baseline"].GetValue()),end='')
    if "ElEl_baseline" in cnt:
        print("\tElEl = {}".format(cnt["ElEl_baseline"].GetValue()),end='')
    if "Mu_baseline" in cnt:
        print("\tMu = {}".format(cnt["Mu_baseline"].GetValue()),end='')
    if "El_baseline" in cnt:
        print("\tEl = {}".format(cnt["El_baseline"].GetValue()),end='')
    print("")
    if "ElMu_baseline" in cnt and "ElEl_baseline" in cnt and "MuMu_baseline" in cnt\
            and "Mu_baseline" in cnt and "El_baseline" in cnt:
        print("\nTotal = {}".format(cnt["ElMu_baseline"].GetValue() + cnt["MuMu_baseline"].GetValue() + cnt["ElEl_baseline"].GetValue() + cnt["Mu_baseline"].GetValue() + cnt["El_baseline"].GetValue()))
    if "selection" in cnt:
        print("Selection = " + str(cnt["selection"].GetValue()))
    else: 
        print("Selection")
    if "ElMu_selection" in cnt:
        print("\tElMu = {}".format(cnt["ElMu_selection"].GetValue()),end='')
    if "MuMu_selection" in cnt:
        print("\tMuMu = {}".format(cnt["MuMu_selection"].GetValue()),end='')
    if "ElEl_selection" in cnt:
        print("\tElEl = {}".format(cnt["ElEl_selection"].GetValue()),end='')
    if "Mu_selection" in cnt:
        print("\tMu = {}".format(cnt["Mu_selection"].GetValue()),end='')
    if "El_selection" in cnt:
        print("\tEl = {}".format(cnt["El_selection"].GetValue()),end='')
    print("")  
    if "ElMu_selection" in cnt and "ElEl_selection" in cnt and "MuMu_selection" in cnt\
            and "Mu_selection" in cnt and "El_selection" in cnt:
        print("\nTotal = {}".format(cnt["ElMu_selection"].GetValue() + cnt["MuMu_selection"].GetValue() + cnt["ElEl_selection"].GetValue() + cnt["Mu_selection"].GetValue() + cnt["El_selection"].GetValue()))
    subfinish[name] = time.clock()
    print("====> Took {}s to process sample {}".format(subfinish[name] - substart[name], name))
finish = time.clock()

===========1/3
tt_DL
Baseline

Selection
	ElMu = 76985
====> Took 128.98s to process sample tt_DL
===========2/3
tttt
Baseline

Selection
	ElMu = 42306
====> Took 78.39s to process sample tttt
===========3/3
ST_tW
Baseline

Selection
	ElMu = 820
====> Took 32.18s to process sample ST_tW


In [20]:
print("Took {}s to process".format(finish - start))
for name, val in substart.items():
    print("Took {}s to process sample {}".format(subfinish[name] - substart[name], name))
print()
masterfinish = time.time() #clock gives cpu time, not accurate multi-core?
print("Took {}m {}s to process in real-time".format((masterfinish - masterstart)//60, (masterfinish - masterstart)%60))

Took 239.55s to process
Took 128.98s to process sample tt_DL
Took 78.39s to process sample tttt
Took 32.18s to process sample ST_tW

Took 6.0m 33.1534380913s to process in real-time


In [28]:
rootDict = {}
histDir = "test_20200123"
if not os.path.isdir(histDir):
    os.makedirs(histDir)
for name, levels_dict in histos1D.items():
    #if "DY" not in name and "t" not in name: continue
    #if theSampleDict[name]["isData"] == True: continue
    print(name, end='')
    #print(theSampleDict[name].keys())
    print(" - c=" + str(theSampleDict[name]["color"]))
    for level, obj_dict in levels_dict.items():
        if level not in levels_of_interest: continue
        print("\t" + level)
        for pre_obj_name, obj_val in obj_dict["Mountains"].items():
            obj_name = "Mountains_" + pre_obj_name
            print("\t\t" + obj_name)
            for hname, hist in obj_val.items():
                print("\t\t\t" + hname)
                #help(hist)
                dictKey = pre_obj_name + "_" + hname
                if dictKey not in rootDict:
                    rootDict[dictKey] = ROOT.TFile.Open("{}.root"\
                                 .format(histDir + "/" + dictKey), "RECREATE")
                rootDict[dictKey].cd()
                hptr = hist.GetPtr()
                oldname = hptr.GetName()
                hptr.SetName("{}".format(name))
                hptr.Write()
                hptr.SetName("{}".format(oldname)) #Avoid overwriting things by switching back, save from segfault
                #hptr.SetFillColor(theSampleDict[name]["color"])
                #hptr.SetLineColor(theSampleDict[name]["color"])
                #stacks[level][obj_name][hname].Add(hptr)
                #stacksource[level][obj_name][hname].append((hptr, hptr.GetIntegral()))
                #Integral fails sometimes, use sum of weights...
                #if theSampleDict[name]["isData"] == False:
                #    stacksource[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
                #else:
                #    stacksource_data[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
        #for obj_name, obj_val in obj_dict.items():
        #    if obj_name == "Mountains": continue #extra depth to account for, custom implementation until next version developed
        #    print("\t\t" + obj_name)
        #    for hname, hist in obj_val.items():
        #        print("\t\t\t" + hname)
        #        #help(hist)
        #        hptr = hist.GetPtr().Clone()
        #        hptr.SetFillColor(theSampleDict[name]["color"])
        #        hptr.SetLineColor(theSampleDict[name]["color"])
        #        #stacks[level][obj_name][hname].Add(hptr)
        #        #stacksource[level][obj_name][hname].append((hptr, hptr.GetIntegral()))
        #        #Integral fails sometimes, use sum of weights...
        #        if theSampleDict[name]["isData"] == False:
        #            stacksource[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
        #        else:
        #            stacksource_data[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
print()
for f in rootDict.values():
    print(type(f))
    f.Close()

tt_DL - c=632
	ElMu_selection
		Mountains_nMediumDeepJet0_nJet7
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_nMediumDeepJet1_nJet8+
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_blind_nMediumDeepJet4+_nJet5
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			

			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_nMediumDeepJet2_nJet6
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_nMediumDeepJet1
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_blind_nJet7
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			

			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_blind_nMediumDeepJet4+_nJet8+
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_blind_nMediumDeepJet3_nJet8+
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_pfRelIso03_all
			dRll
			dEtall
			dRbb
			MTofMETandEl
			MET_pt
			dEtabb
			nMediumDeepCSV
			HTRat
			HTb
			Electron_pfRelIso03_chg
			H
			MET_phi
			MTofElandMu
			Muon_pfRelIso03_all
			HTH
			H2M
			nMediumDeepJet
			dPhill
		Mountains_blind_nJet8+
			Muon_pfRelIso03_chg
			Muon_pfRelIso04_all
			HT2M
			HT
			MTofMETandMu
			dPhibb
			Electron_

<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.

<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.TFile'>
<class 'ROOT.

In [ ]:
stacks = {}
stacksource = {} #Create sortable lists to fill stacks from
stacksource_data = {} #create separte list to append all the data to, so that they can be conbined into one hist file and added to the stacksoure at the end
model_dict = [histos1D[k] for k in theSampleDict.keys() if theSampleDict[k]["isData"] == False]
model_dict = model_dict[0]
if len(model_dict) < 1:
    raise RuntimeError("Failure, no histogram dictionary found to form stacks from")
for level, obj_dict in model_dict.items():
    if level not in levels_of_interest: continue
    stacks[level] = {}
    stacksource[level] = {}
    stacksource_data[level] = {}
    for obj_name, obj_val in obj_dict.items():
        if obj_name == "Mountains": continue #extra depth to account for, custom implementation until next version developed
        stacks[level][obj_name] = {}
        stacksource[level][obj_name] = {}
        stacksource_data[level][obj_name] = {}
        for hname, hist in obj_val.items():
            stacks[level][obj_name][hname] = []
            stacks[level][obj_name][hname].append(ROOT.THStack("s_{}_{}_{}".format(level, obj_name, hname), "{}_{}_{}".format(level, obj_name, hname)))
            stacksource[level][obj_name][hname] = []
            stacksource_data[level][obj_name][hname] = []
    for pre_obj_name, obj_val in obj_dict["Mountains"].items():
        obj_name = "Mountains_" + pre_obj_name
        stacks[level][obj_name] = {}
        stacksource[level][obj_name] = {}
        stacksource_data[level][obj_name] = {}
        for hname, hist in obj_val.items():
            stacks[level][obj_name][hname] = []
            stacks[level][obj_name][hname].append(ROOT.THStack("s_{}_{}_{}".format(level, obj_name, hname), "{}_{}_{}".format(level, obj_name, hname)))
            stacksource[level][obj_name][hname] = []
            stacksource_data[level][obj_name][hname] = []
for name, levels_dict in histos1D.items():
    #if "DY" not in name and "t" not in name: continue
    #if theSampleDict[name]["isData"] == True: continue
    print(name, end='')
    #print(theSampleDict[name].keys())
    print(" - c=" + str(theSampleDict[name]["color"]))
    for level, obj_dict in levels_dict.items():
        if level not in levels_of_interest: continue
        print("\t" + level)
        for pre_obj_name, obj_val in obj_dict["Mountains"].items():
            obj_name = "Mountains_" + pre_obj_name
            print("\t\t" + obj_name)
            for hname, hist in obj_val.items():
                print("\t\t\t" + hname)
                #help(hist)
                hptr = hist.GetPtr().Clone()
                hptr.SetFillColor(theSampleDict[name]["color"])
                hptr.SetLineColor(theSampleDict[name]["color"])
                #stacks[level][obj_name][hname].Add(hptr)
                #stacksource[level][obj_name][hname].append((hptr, hptr.GetIntegral()))
                #Integral fails sometimes, use sum of weights...
                if theSampleDict[name]["isData"] == False:
                    stacksource[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
                else:
                    stacksource_data[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
        for obj_name, obj_val in obj_dict.items():
            if obj_name == "Mountains": continue #extra depth to account for, custom implementation until next version developed
            print("\t\t" + obj_name)
            for hname, hist in obj_val.items():
                print("\t\t\t" + hname)
                #help(hist)
                hptr = hist.GetPtr().Clone()
                hptr.SetFillColor(theSampleDict[name]["color"])
                hptr.SetLineColor(theSampleDict[name]["color"])
                #stacks[level][obj_name][hname].Add(hptr)
                #stacksource[level][obj_name][hname].append((hptr, hptr.GetIntegral()))
                #Integral fails sometimes, use sum of weights...
                if theSampleDict[name]["isData"] == False:
                    stacksource[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
                else:
                    stacksource_data[level][obj_name][hname].append((hptr, hptr.GetSumOfWeights(), theSampleDict[name]["isData"]))
print()
#Now cycle through and sort each list, once it contains all hists from every source (outermost loop - name - above)
print(stacksource_data)
for level, obj_dict in model_dict.items():
    if level not in levels_of_interest: continue
    for pre_obj_name, obj_val in obj_dict["Mountains"].items():
        obj_name = "Mountains_" + pre_obj_name
        for hname, hist in obj_val.items():
            #Sort the MC-only histograms
            stacksource[level][obj_name][hname].sort(key=lambda b: b[1], reverse=False)
            
            #Create a MC-only histogram for statistics purposes
            tmpMC = None
            for himc, h_mc in enumerate(stacksource[level][obj_name][hname]):
                if himc == 0:
                    #take first histo
                    tmpMC = h_mc[0].Clone()
                    tmpMC.SetTitle("MC")
                else:
                    #hadd the other histos
                    #tmpMC = tmpMC + h_mc[0].Clone()
                    tmpMC.Add(h_mc[0].Clone())
            if tmpMC != None:
                #tmpMC.SetMarkerStyle(0)
                tmpMC.SetLineColor(ROOT.kRed)  #FIXME Color from largest sample?
                tmpMC.SetFillColorAlpha(ROOT.kRed, 0) #FIXME
            stacks[level][obj_name][hname].append(tmpMC)
            
            tmp = None
            for hid, h_data in enumerate(stacksource_data[level][obj_name][hname]):
                if hid == 0:
                    #take first histo
                    tmp = h_data[0].Clone()
                    tmp.SetTitle("Data")
                else:
                    #hadd the other histos
                    #tmp = tmp + h_data[0].Clone()
                    tmp.Add(h_data[0].Clone())
            if tmp != None:
                tmp.SetMarkerStyle(0) #20 round dot, with SetMarkerSize(1.0) in an example
                tmp.SetLineColor(ROOT.kBlack)
                tmp.SetFillColorAlpha(ROOT.kWhite, 0)
            stacks[level][obj_name][hname].append(tmp)
                
            for hptrTup in stacksource[level][obj_name][hname]:
                #add to the THStack in the first position of the tuple
                stacks[level][obj_name][hname][0].Add(hptrTup[0])
    for obj_name, obj_val in obj_dict.items():
        if obj_name == "Mountains": continue #extra depth to account for, custom implementation until next version developed
        for hname, hist in obj_val.items():
            #Sort the MC-only histograms
            stacksource[level][obj_name][hname].sort(key=lambda b: b[1], reverse=False)
            
            #Create a MC-only histogram for statistics purposes
            tmpMC = None
            for himc, h_mc in enumerate(stacksource[level][obj_name][hname]):
                if himc == 0:
                    #take first histo
                    tmpMC = h_mc[0].Clone()
                    tmpMC.SetTitle("MC")
                else:
                    #hadd the other histos
                    #tmpMC = tmpMC + h_mc[0].Clone()
                    tmpMC.Add(h_mc[0].Clone())
            if tmpMC != None:
                #tmpMC.SetMarkerStyle(0)
                tmpMC.SetLineColor(ROOT.kRed) #FIXME Color set to highest integral's
                tmpMC.SetLineWidth(0)
                tmpMC.SetFillColorAlpha(ROOT.kRed, 0)
            stacks[level][obj_name][hname].append(tmpMC)
            
            tmp = None
            for hid, h_data in enumerate(stacksource_data[level][obj_name][hname]):
                if hid == 0:
                    #take first histo
                    tmp = h_data[0].Clone()
                    tmp.SetTitle("Data")
                else:
                    #hadd the other histos
                    #tmp = tmp + h_data[0].Clone()
                    tmp.Add(h_data[0].Clone())
            if tmp != None:
                tmp.SetMarkerStyle(0) #20 round dot, with SetMarkerSize(1.0) in an example
                tmp.SetLineColor(ROOT.kBlack)
                #tmp.SetFillColorAlpha(ROOT.kWhite, 0)
            stacks[level][obj_name][hname].append(tmp)
                
            for hptrTup in stacksource[level][obj_name][hname]:
                #add to the THStack in the first position of the tuple
                stacks[level][obj_name][hname][0].Add(hptrTup[0])

In [ ]:
#c2 = ROOT.TCanvas()
#c2.cd()
#histos2D["tttt"]["ElMu_selection"]["Mountains"]["nMediumDeepCSV0"]["npvsGood_vs_nTrueInt"].Draw("COLZ")
#c2.Draw()

In [ ]:
leg = ROOT.TLegend(0.75,0.80, 0.95, 0.90)
#leg = ROOT.TLegend(0.15,0.10)
#leg.SetFillColor(0)
#leg.SetBorderSize(0)
leg.SetNColumns(2)
leg.SetTextSize(0.03)
leg_colors = set([smpl["color"] for smpl in theSampleDict.values()])
leg_tuple = [(smpl[0], smpl[1]) for smpl in leg_dict.items() if smpl[1] in leg_colors]
leg_hists = {}
for samplecategory, color in leg_tuple:
    leg_hists[color] = ROOT.TH1D(samplecategory, samplecategory, 0, 0, 1)
    if samplecategory != "Data":
        leg_hists[color].SetFillColor(color)
        leg.AddEntry(leg_hists[color], samplecategory, "F")
    else:
        leg.AddEntry(leg_hists[color], samplecategory, "P")
%jsroot on 
#help(ROOT.TLegend)

In [ ]:
%jsroot off
for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if obj_name != "Jets": continue
    print(obj_name)
    for sname, stack in obj_dict.items():
        c = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
        c.cd()
        c.Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        stack[0].Draw("HIST S")
        #The `mode` has up to nine digits that can be set to on (1 or 2), off (0).
 
         #mode = ksiourmen  (default = 000001111)
         #k = 1;  kurtosis printed
         #k = 2;  kurtosis and kurtosis error printed
         #s = 1;  skewness printed
         #s = 2;  skewness and skewness error printed
         #i = 1;  integral of bins printed
         #i = 2;  integral of bins with option "width" printed
         #o = 1;  number of overflows printed
         #u = 1;  number of underflows printed
         #r = 1;  standard deviation printed
         #r = 2;  standard deviation and standard deviation error printed
         #m = 1;  mean value printed
         #m = 2;  mean and mean error values printed
         #e = 1;  number of entries printed
         #n = 1;  name of histogram is printed
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[1].Draw("SAMES")
        if len(stack) > 2 and stack[2] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Preliminary}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        c.Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None:
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")
        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        c.SetLogy()
        c.SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        c.SetLogy(0)
        c.SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        

In [ ]:
for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if obj_name != "Muons": continue
    print(obj_name)
    for sname, stack in obj_dict.items():
        c = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
        c.cd()
        c.Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        stack[0].Draw("HIST S")
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[1].Draw("SAMES")
        if len(stack) > 2 and stack[2] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Preliminary}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        c.Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None:
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")
        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        c.SetLogy()
        c.SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        c.SetLogy(0)
        c.SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        

In [ ]:
for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if obj_name != "Electrons": continue
    print(obj_name)
    for sname, stack in obj_dict.items():
        c = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
        c.cd()
        c.Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        stack[0].Draw("HIST S")
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[1].Draw("SAMES")
        if len(stack) > 2 and stack[2] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Preliminary}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        c.Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None:
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")
        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        c.SetLogy()
        c.SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        c.SetLogy(0)
        c.SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        

In [ ]:
for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if obj_name != "Leptons": continue
    print(obj_name)
    for sname, stack in obj_dict.items():
        c = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
        c.cd()
        c.Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        stack[0].Draw("HIST S")
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[1].Draw("SAMES")
        if len(stack) > 2 and stack[2] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Preliminary}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        c.Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None:
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")
        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        c.SetLogy()
        c.SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        c.SetLogy(0)
        c.SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        

In [ ]:
for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if obj_name != "EventVars": continue
    print(obj_name)
    for sname, stack in obj_dict.items():
        c = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
        c.cd()
        c.Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        stack[0].Draw("HIST S")
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[1].Draw("SAMES")
        if len(stack) > 2 and stack[2] != None:
            ROOT.gStyle.SetOptStat(1111111)
            stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Preliminary}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        c.Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None:
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")
        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        c.SetLogy()
        c.SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        c.SetLogy(0)
        c.SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        

In [ ]:
#%jsroot on
unblind_whitelist = set([])
CanvasCache = {}
CanvasCache["Open/Close"] = ROOT.TCanvas("open_close", "", 800, 100)

for obj_name, obj_dict in stacks["ElMu_selection"].items():
    if "Mountains" not in obj_name: continue
    print(obj_name)
    CanvasCache[obj_name] = {}
    #Save to a pdf using the '.pdf(' string to make a file that stays open for subsequent writes to the same filename. To be closed by '.pdf)' 
    CanvasCache["Open/Close"].SaveAs("./{channel}/{object_name}_All.pdf(".format(channel=fileChannel.replace("_selection", ""), object_name=obj_name.replace("Mountains_", "")))
    for sname, stack in sorted(obj_dict.items()):
        c = None
        if "_vs_" in sname: #hack to draw 2D histos in the 1D histo dict
            CanvasCache[obj_name][sname] = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 1800)
            CanvasCache[obj_name][sname].Divide(1,3)
            CanvasCache[obj_name][sname].cd(1)
        else:
        #if True:
            CanvasCache[obj_name][sname] = ROOT.TCanvas("cs_{}_{}".format(obj_name, sname), "", 800, 600)
            CanvasCache[obj_name][sname].cd()
        CanvasCache[obj_name][sname].SetLogy()
        CanvasCache[obj_name][sname].Update()
        #For data first
        #if len(stack) > 1:
        #    stack[1].Draw("PE1")
        #    stack[0].Draw("HIST SAME")
        #else:
        #    stack[0].Draw("HIST")
        #if len(stack) > 1:
        #    stack[1].Draw("PE1 SAME")
        #for MC first
        if "_vs_" in sname: #hack to draw 2D histos in the 1D histo dict
            stack[0].Draw("")
        else:
            stack[0].Draw("HIST S")
        #Draw the MC summed histogram for stats (better way with THStack? why the fuck don't people document this in a useful way?)
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1111111)
            if "_vs_" in sname: #hack to draw 2D histos in the 1D histo dict
                stack[1].Draw("SAMES") #Also SAME0, SAMES0 which 'do not use the z axis of the previous plot'
                CanvasCache[obj_name][sname].cd(2)
                tmp1 = stack[1].Clone()
                tmp1.SetLineColor(ROOT.kBlue)
                tmp1.SetFillColorAlpha(ROOT.kGreen, 0.7)
                tmp1.Draw("VIOLINX")
                #stack[1].ProfileX().Draw("E3")
                CanvasCache[obj_name][sname].cd(3)
                tmp1.Draw("VIOLINY")
                #stack[1].ProfileY().Draw("E3")
                CanvasCache[obj_name][sname].cd(1)
            else:
                stack[1].Draw("SAMES HIST")
        #Draw the data histogram, assuming that it's unblinded ("blind" not in the name and or in the whitelist )
        if len(stack) > 2 and stack[2] != None and ("blind" not in obj_name or obj_name in unblind_whitelist):
            ROOT.gStyle.SetOptStat(1111111)
            if "_vs_" in sname: #hack to draw 2D histos in the 1D histo dict
                stack[2].Draw("SAMES") #Maybe add ARR for arrow mode? or something else
                CanvasCache[obj_name][sname].cd(2)
                tmp2 = stack[2].Clone()
                tmp2.SetLineColor(ROOT.kRed)
                tmp2.SetFillColorAlpha(ROOT.kGray, 0.4)
                tmp2.Draw("SAMES CANDLEX")
                #stack[2].ProfileX().Draw("PE1 SAMES")
                CanvasCache[obj_name][sname].cd(3)
                tmp2.Draw("SAMES CANDLEY")
                #stack[2].ProfileY().Draw("PE1 SAMES")
                CanvasCache[obj_name][sname].cd(1)
            else:
                stack[2].Draw("PE1 SAMES")
        # Add header
        cms_label = ROOT.TLatex()
        cms_label.SetTextSize(0.04)
        cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Internal}")
        header = ROOT.TLatex()
        header.SetTextSize(0.03)
        header.DrawLatexNDC(0.63, 0.92, "#sqrt{{s}} = 13 TeV, L_{{int}} = {0} fb^{{-1}}".format(lumi[era]))
        leg.Draw()
        CanvasCache[obj_name][sname].Draw()
        if len(stack) > 1 and stack[1] != None:
            ROOT.gStyle.SetOptStat(1001111)
            StatBox = stack[1].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.7)
            StatBox.SetY2NDC(0.5)
            #StatBox.SetName("MC")
        if len(stack) > 2 and stack[2] != None and ("blind" not in obj_name or obj_name in unblind_whitelist):
            #ROOT.gStyle.SetOptStat(111111)
            StatBox = stack[2].GetListOfFunctions().FindObject("stats")
            StatBox.SetY1NDC(0.5)
            StatBox.SetY2NDC(0.3)
            #StatBox.SetLabel("Data")

        if not os.path.isdir("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name)):
            os.makedirs("./{channel}/{object_name}/".format(channel=fileChannel, object_name=obj_name))
        CanvasCache[obj_name][sname].SetLogy()
        CanvasCache[obj_name][sname].SaveAs("./{channel}/{object_name}/{hname}_LOGY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        CanvasCache[obj_name][sname].SaveAs("./{channel}/{object_name}_All.pdf".format(channel=fileChannel.replace("_selection", ""), object_name=obj_name.replace("Mountains_", ""), wgtVar=theWeight, hname=sname, filetype=theFormat))
        CanvasCache[obj_name][sname].SetLogy(0)
        CanvasCache[obj_name][sname].SaveAs("./{channel}/{object_name}/{hname}_LINY{filetype}".format(channel=fileChannel, object_name=obj_name, wgtVar=theWeight, hname=sname, filetype=theFormat))
        CanvasCache[obj_name][sname].SaveAs("./{channel}/{object_name}_All.pdf".format(channel=fileChannel.replace("_selection", ""), object_name=obj_name.replace("Mountains_", ""), wgtVar=theWeight, hname=sname, filetype=theFormat))
    #Close the pdf using '.pdf)' 
    CanvasCache["Open/Close"].SaveAs("./{channel}/{object_name}_All.pdf)".format(channel=fileChannel.replace("_selection", ""), object_name=obj_name.replace("Mountains_", "")))
    

In [ ]:
masterfinish2 = time.time() #clock gives cpu time, not accurate multi-core?
print("Took {}m {}s to process in real-time including plots".format((masterfinish2 - masterstart)//60, (masterfinish - masterstart)%60))

In [ ]:
#From example: https://root.cern.ch/doc/master/df103__NanoAODHiggsAnalysis_8py_source.html

#def plot(sig, bkg, data, x_label, filename):
#     """
#     Plot invariant mass for signal and background processes from simulated
#     events overlay the measured data.
#     """
#     # Canvas and general style options
#     ROOT.gStyle.SetOptStat(0)
#     ROOT.gStyle.SetTextFont(42)
#     d = ROOT.TCanvas("d", "", 800, 700)
#     d.SetLeftMargin(0.15)
# 
#     # Get signal and background histograms and stack them to show Higgs signal
#     # on top of the background process
#     h_bkg = bkg
#     h_cmb = sig.Clone()
# 
#     h_cmb.Add(h_bkg)
#     h_cmb.SetTitle("")
#     h_cmb.GetXaxis().SetTitle(x_label)
#     h_cmb.GetXaxis().SetTitleSize(0.04)
#     h_cmb.GetYaxis().SetTitle("N_{Events}")
#     h_cmb.GetYaxis().SetTitleSize(0.04)
#     h_cmb.SetLineColor(ROOT.kRed)
#     h_cmb.SetLineWidth(2)
#     h_cmb.SetMaximum(18)
#     h_bkg.SetLineWidth(2)
#     h_bkg.SetFillStyle(1001)
#     h_bkg.SetLineColor(ROOT.kBlack)
#     h_bkg.SetFillColor(ROOT.kAzure - 9)
# 
#     # Get histogram of data points
#     h_data = data
#     h_data.SetLineWidth(1)
#     h_data.SetMarkerStyle(20)
#     h_data.SetMarkerSize(1.0)
#     h_data.SetMarkerColor(ROOT.kBlack)
#     h_data.SetLineColor(ROOT.kBlack)
# 
#     # Draw histograms
#     h_cmb.DrawCopy("HIST")
#     h_bkg.DrawCopy("HIST SAME")
#     h_data.DrawCopy("PE1 SAME")
# 
#     # Add legend
#     legend = ROOT.TLegend(0.62, 0.70, 0.82, 0.88)
#     legend.SetFillColor(0)
#     legend.SetBorderSize(0)
#     legend.SetTextSize(0.03)
#     legend.AddEntry(h_data, "Data", "PE1")
#     legend.AddEntry(h_bkg, "ZZ", "f")
#     legend.AddEntry(h_cmb, "m_{H} = 125 GeV", "f")
#     legend.Draw()
# 
#     # Add header
#     cms_label = ROOT.TLatex()
#     cms_label.SetTextSize(0.04)
#     cms_label.DrawLatexNDC(0.16, 0.92, "#bf{CMS Open Data}")
#     header = ROOT.TLatex()
#     header.SetTextSize(0.03)
#     header.DrawLatexNDC(0.63, 0.92, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")
# 
#     # Save plot
#     d.SaveAs(filename)

In [ ]:
#gg1 = ROOT.ROOT.RDF.SaveGraph(the_df['tttt']['ElMu_selection'], './mydot.dot')
#!dot -Tsvg mydot.dot -o mydot.svg
#listOfImageNames = ['./mydot.svg',
#                    ]
#
#for imageName in listOfImageNames:
#    display(SVG(filename=imageName))

In [ ]:
#c = ROOT.TCanvas()


In [ ]:
#stacks["ElMu_selection"]["Mountains_nMediumDeepJet0"]["Muon_pfRelIso03_chg_vs_MET"]
#stacksource_data["ElMu_selection"]["Mountains_nMediumDeepJet0"]["Muon_pfRelIso03_chg_vs_MET"]